### **Code to extract trips between two locations**

Split into three parts:
1. The first part includes breaking down the bigger file into smaller chunks (`TRIP_DETAIL`)
2. Finding numberplate passing through location1 and location 2 on the same date (done using `platedate`)
3. Using those platedates to filter out trips between the two locations

**Note**: Since I've broken down TRIP_DETAIL into smaller files, we need to run the code for each file 

In [ ]:

%%capture
import pandas as pd
# import modin.pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt
import vaex # https://vaex.io/docs/index.html
import pathlib
from pathlib import *
import os
import pickle
# import cufflinks as cf
# import chart_studio.plotly as py
import seaborn as sns
import plotly.express as px
%matplotlib inline
from plotly.offline import download_plotlyjs, init_notebook_mode,plot,iplot

init_notebook_mode(connected=True)
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

### **Part I**

In [ ]:
##TODO -> Create functions for repetitive tasks

# ,skiprows=range(3, 260000000)
# Input the csv
# Extracting a subset of 1000000 rows by default
def load_data(dir_name,base_filename):
    complete_path=os.path.join(dir_name, base_filename + "." + "csv")
    # df=pd.read_csv(complete_path,sep=';',encoding= 'unicode_escape',nrows=10000,engine='c',infer_datetime_format=True,usecols=['Trip_Summary_Id','Numberplate','Point_time-stamp','Latitude',
    #                                                                                                                           'Longitude','Meters_travelled','Time_elapsed','Point_speed','Number_of_lanes',
    #                                                                                                                           'Road_type','Road_class','Road_form','Speed_restriction','TNO_Time-stamp'])
    # df = pd.concat((chunk for chunk in pd.read_csv(complete_path,sep=';',nrows=5000000,encoding= 'unicode_escape',engine='c',infer_datetime_format=True,chunksize=1000000,low_memory=False,usecols=['Numberplate','Latitude',
    #                                                                                                                           'Longitude','Meters_travelled','Time_elapsed','Point_speed','Number_of_lanes',
    #                                                                                                                           'Road_type','Road_class','Speed_restriction','TNO_Time-stamp'])))
    
    df = pd.concat((chunk for chunk in pd.read_csv(complete_path,sep=';',nrows=20000000,encoding= 'unicode_escape',engine='c',infer_datetime_format=True,chunksize=1000000,low_memory=False)))
    return df

# Dropping the first and last row of csv ("------")
def drop_first_row(df):
    df=df.iloc[1:]
    df=df[:-1]
    return df

def resetIndex(df):
    return df.reset_index()


##TODO -> Rewrite this function

def cast_to_correct_dtype(df):
    
    if 'Id' in df.columns:
        df['Id'] = df['Id'].astype('int')
        
    if 'AOS_position_Id' in df.columns:
        df['AOS_position_Id'] = df['AOS_position_Id'].astype('int')
    
    if 'Acceleration_x' in df.columns:
        df['Acceleration_x'] = df['Acceleration_x'].astype('float')
        
    if 'Acceleration_y' in df.columns:
        df['Acceleration_y'] = df['Acceleration_y'].astype('float')
        
    if 'TNO_Valid' in df.columns:
        df['TNO_Valid'] = df['TNO_Valid'].astype('int')
    
    if 'Latitude' in df.columns:
        df['Latitude'] = df['Latitude'].astype('float')
        
    if 'Longitude' in df.columns:
        df['Longitude'] = df['Longitude'].astype('float')
        
    if 'Event/action_speed' in df.columns:
        df['Event/action_speed'] = df['Event/action_speed'].astype('int')
        
    if 'Event/action_type' in df.columns:
        df['Event/action_type'] = df['Event/action_type'].astype('int')    
        
    if 'Number_of_lanes' in df.columns:
        df['Number_of_lanes'] = df['Number_of_lanes'].astype('int')
        
    if 'Road_class' in df.columns:
        df['Road_class'] = df['Road_class'].astype('int')
        
    if 'Road_type' in df.columns:
        df['Road_type'] = df['Road_type'].astype('int')
        
    if 'Crash_position_Id' in df.columns:
        df['Crash_position_Id']=df['Crash_position_Id'].astype('int')
        
    if 'Crash_position_Id' in df.columns:
        df['Crash_position_Id'] = df['Crash_position_Id'].astype('int')
        
    if 'Point_speed' in df.columns:
        df['Point_speed'] = df['Point_speed'].astype('float')
        
    if 'Average_speed_fpp' in df.columns:
        df['Average_speed_fpp'] = df['Average_speed_fpp'].astype('float')  
        
    if 'Average_Speed' in df.columns:
        df['Average_Speed'] = df['Average_Speed'].astype('float')        

    if 'Maximum_speed' in df.columns:
        df['Maximum_speed'] = df['Maximum_speed'].astype('float')     
        
    if 'Meters_travelled' in df.columns:
        df['Meters_travelled'] = df['Meters_travelled'].astype('int')  

    if 'Road_form' in df.columns:
        df['Road_form'] = df['Road_form'].astype('int') 
        
    if 'Speed_restriction' in df.columns:
        df['Speed_restriction'] = df['Speed_restriction'].astype('int') 
        
    if 'Crash_speed' in df.columns:
        df['Crash_speed'] = df['Crash_speed'].astype('int')
        
    if 'Maximum_acceleration' in df.columns:
        df['Maximum_acceleration'] = df['Maximum_acceleration'].astype('float')
        
    if 'Numberplate' in df.columns:
        df['Numberplate']=df['Numberplate'].astype('str')
        

    return df

            

## Handling date time related fields

def cast_date_time(df):
    if 'AOS_event/action_time' in df.columns:
        df['AOS_event/action_time']=pd.datetime(df['AOS_event/action_time'],errors='coerce')
        df['AOS_event/action_time_hour']=df['AOS_event/action_time'].dt.hour
    
    if 'TNO_Trip-start' in df.columns:
        df['TNO_Trip-start'] = pd.to_datetime(df['TNO_Trip-start'],errors='coerce')
        df['TNO_Trip-start_hour'] = df['TNO_Trip-start'].dt.hour
        
    if 'TNO_Trip-end' in df.columns:
        df['TNO_Trip-end'] = pd.to_datetime(df['TNO_Trip-end'],errors='coerce')
        
    if 'Position_time' in df.columns:
        df['Position_time'] = pd.to_datetime(df['Position_time'],errors='coerce')

    if 'TNO_Time-stamp' in df.columns:
        df['TNO_Time-stamp'] = pd.to_datetime(df['TNO_Time-stamp'],errors='coerce')
        df['TNO_Time-stamp_hour'] = df['TNO_Time-stamp'].dt.hour
        
    return df

## Converting Time Stamps to datetime

def date_and_time_columns(df):
    if 'AOS_event/action_time' in df.columns:
        df['AOS Trip Date']=df['AOS_event/action_time'].dt.date
        df['AOS Event Time']=df['AOS_event/action_time'].dt.time
        df['AOS Trip Hour']=df['AOS_event/action_time'].dt.hour
    
    
    if 'TNO_Time-stamp' in df.columns:
        df['TNO Trip Date'] = df['TNO_Time-stamp'].dt.date
        df['TNO Trip Time'] = df['TNO_Time-stamp'].dt.time
        df['TNO Trip Hour']=df['TNO_Time-stamp'].dt.hour
        # df['TNO Trip Hour'] = df['TNO Trip Time'].dt.hour
    return df

def Time_of_the_day(x):
    if (x > 4) and (x <= 8):
        return 'Early Morning'
    elif (x > 8) and (x < 12 ):
        return 'Morning'
    elif (x >= 12) and (x <= 16):
        return'Noon'
    elif (x > 16) and (x <= 20) :
        return 'Evening'
    elif (x > 20) and (x <= 24):
        return'Night'
    elif (x <= 4):
        return'Late Night'
    
    
def rename_some_stuff(df):
    if 'Event/action_type' in df.columns:
        df["Event type Rename"]=df["Event/action_type"].replace({0: 'Headway Warning = OFF', 1: 'Headway Warning (long)', 2: 'Headway Warning (medium)'
                                                                                 ,3: 'Headway Warning (short)',10:'Lane Departure Warning = OFF',11:'Left Lane Departure Warning = ON'
                                                                                 ,12:'Right Lane Departure Warning = ON',13:'Left and Right Lane Departure Warning = ON'
                                                                                 ,20:' Indicators = OFF',21:'Left Indicator = ON',22:'Right Indicator = ON',23:'Left and Right Indicator = ON'
                                                                                 ,40:'Brakes = OFF',41:'Brakes = ON'})
        
    if 'Road_type' in df.columns:
        df["Road_type_Rename"]=df["Road_type"].replace({0: "Urban", 1: "Motorway",2:"Extra Urban",3:"Unavailable"})
        
    return df

def detect_overspeeding_count(df):
    # create a list of our conditions
    # if 'Event/action_speed' and 'Speed_restriction' in df.columns:
    #     conditions = [(df['Event/action_speed']> df['Speed_restriction'])]
    #     # # create a list of the values we want to assign for each condition
    #     values = [1]
    #     # create a new column and use np.select to assign values to it using our lists as arguments
    #     df['Overspeeding_event'] = np.select(conditions, values)
        
    # return df

    if 'Point_speed' and 'Speed_restriction' in df.columns:
        conditions = [(df['Point_speed']> df['Speed_restriction'])]
        # # create a list of the values we want to assign for each condition
        values = [1]
        # create a new column and use np.select to assign values to it using our lists as arguments
        df['Overspeeding_event'] = np.select(conditions, values)
        
    return df

def some_processing(df):
    if 'Meters_travelled' in df.columns:
        df['KmTravelled']=df['Meters_travelled']/1000
    return df

In [ ]:
df20M=pd.read_pickle(r'D:\AOS FOT\Octo\pickle_files\Complete_data_Trip_detail\TRIP_DETAIL_20M.pkl')
df20M
# df20M['Speed_restriction'].unique()

In [ ]:
a=df20M['Admin_area'].value_counts()

#### Saving a TRIP_DETAIL/AOS_DETAIL files as smaller pkl files and CSVs  

In [ ]:
# pd.read_csv(r'D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\AOS_SUMMARY.csv',nrows=200000,error_bad_lines=False,delimiter=';')

In [ ]:
%%capture
df_TRIP_DETAIL = (load_data("D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY","AOS_SUMMARY")
                                      .pipe(drop_first_row)
                                      .pipe(resetIndex)
                                      .pipe(cast_to_correct_dtype)
                                      .pipe(cast_date_time) # TODO some error here that needs to be resolved later
                                      .pipe(date_and_time_columns)
                                      .pipe(rename_some_stuff)
                                      .pipe(detect_overspeeding_count))

df_TRIP_DETAIL['AOS_event/action_time'] = pd.to_datetime(df_TRIP_DETAIL['AOS_event/action_time'], format= '%H:%M:%S', errors='ignore')
df_TRIP_DETAIL.info
# df_TRIP_DETAIL['AOS_event/action_time']=pd.datetime(df_TRIP_DETAIL['AOS_event/action_time'])
# df_TRIP_DETAIL['AOS_event/action_date']=df_TRIP_DETAIL['AOS_event/action_time'].dt.date
# df_TRIP_DETAIL['Part of Day'] = df_TRIP_DETAIL['AOS_event/action_time'].apply(Time_of_the_day)
df_TRIP_DETAIL['AOS_event/action_date_str'] = df_TRIP_DETAIL['AOS_event/action_date'].apply(lambda x: str(x))
df_TRIP_DETAIL['platedate'] = df_TRIP_DETAIL['Numberplate'] + df_TRIP_DETAIL['AOS_event/action_date_str'] # Adding platedate is important cause that's how we'll filter trips later
df_TRIP_DETAIL.info(verbose=False, memory_usage="deep")

In [ ]:
df_TRIP_DETAIL.info()

### Saving dataframe as pickle file and CSV

Note: You don't need to do all these steps I can share the saved files with you directly if you want

In [ ]:
df_TRIP_DETAIL.to_pickle(r'D:\AOS FOT\Octo\pickle_files\Complete_data_aos_summary\AOS_SUMMARY_80M.pkl')

In [ ]:
df_TRIP_DETAIL.to_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\AOS_SUMMARY_80M.csv')

### Reading Pickle Files

In [ ]:
df_TRIP_DETAIL_10M=pd.read_pickle(r'D:\AOS FOT\Octo\pickle_files\Complete_data_Trip_detail\TRIP_DETAIL_20M.pkl')
df_TRIP_DETAIL_10M

In [ ]:
a=df_TRIP_DETAIL_10M['Admin_area'].value_counts()
a

In [ ]:
df_TRIP_DETAIL_10M = df_TRIP_DETAIL_10M.drop(df_TRIP_DETAIL_10M[df_TRIP_DETAIL_10M['Point_speed']==0].index)

In [ ]:
df_TRIP_DETAIL_10M_mway = df_TRIP_DETAIL_10M[df_TRIP_DETAIL_10M['Road_type_Rename']=='Motorway']

In [ ]:
df_TRIP_DETAIL_10M_mway = df_TRIP_DETAIL_10M_mway.drop(df_TRIP_DETAIL_10M_mway[df_TRIP_DETAIL_10M_mway['Point_speed']>150].index)

In [ ]:
import seaborn as sns
# tips = sns.load_dataset("tips")
ax = sns.boxplot(x=df_TRIP_DETAIL_10M_mway["Point_speed"])

In [ ]:
df_TRIP_DETAIL_10M_mway_grouped=df_TRIP_DETAIL_10M_mway.groupby('Numberplate').mean() 
# df_TRIP_DETAIL_10M_mway_grouped

In [ ]:
import seaborn as sns
# tips = sns.load_dataset("tips")
ax = sns.boxplot(x=df_TRIP_DETAIL_10M_mway_grouped["Point_speed"])

In [ ]:
df_AOS_SUMMARY_20M=pd.read_pickle(r'D:\AOS FOT\Octo\pickle_files\Complete_data_aos_summary\aos_summary_20M.pkl')
df_AOS_SUMMARY_20M.columns

In [ ]:
df_AOS_SUMMARY_20M

### **Part II**

### Extracting Common Platedates

- In this step we'll extract the `platedate` that passed through `area_list = ['start_location','end_location']` on the same day.

- We'll save these values as a list and use it later to extract trips.

In [ ]:
location1='Utrecht' # Define Location 1 here
location2='Eindhoven' # Define Location 2 here

In [ ]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar



area_list = [location1,location2] # We can change these values to any start and end location
df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\TRIP_DETAIL.csv',delimiter=';', encoding='unicode_escape', usecols=['Id', 'TNO_Time-stamp', 'Numberplate', 'Admin_area'])

df = df[df['Admin_area'].isin(area_list)]
with ProgressBar():
    df = df.compute()
    
df['platedate'] = df['Numberplate'] + df['TNO_Time-stamp'].str[:10]

In [ ]:
## Casting `TNO_Time-stamp` to correct dtype
df['TNO_Time-stamp'] = pd.to_datetime(df['TNO_Time-stamp'],errors='coerce')
def date_and_time_columns(df):
    if 'TNO_Time-stamp' in df.columns:
        df['TNO Trip Date'] = df['TNO_Time-stamp'].dt.date
    return df

df=date_and_time_columns(df)
df['TNO_date_str'] = df['TNO Trip Date'].apply(lambda x: str(x))

In [ ]:
df_location1=df[(df['Admin_area']==location1)] # We can replace these locations to anything else
df_location2=df[(df['Admin_area']==location2)]

df_location1=cast_to_correct_dtype(df_location1)
df_location2=cast_to_correct_dtype(df_location2)

In [ ]:
common_utrecht_leek_numberplate=(np.intersect1d(df_location1['Numberplate'], df_location2['Numberplate']))
common_utrecht_leek_platedate=(np.intersect1d(df_location1['platedate'], df_location2['platedate']))
# common_utrecht_leek_numberplate

In [ ]:
# Saving common platedate list as pkl file

with open("common_utrecht_eindhoven_numberplate", "wb") as fp:   #Pickling
    pickle.dump(common_utrecht_leek_numberplate, fp)
    
with open("common_utrecht_eindhoven_platedate", "wb") as fp:   #Pickling
    pickle.dump(common_utrecht_leek_platedate, fp)

In [ ]:
df_utrecht_points = df_location1[df_location1['Numberplate'].isin(common_utrecht_leek_numberplate)]
df_leek_points = df_location2[df_location2['Numberplate'].isin(common_utrecht_leek_numberplate)]
# df_utrecht_points

### **Part III**

In [ ]:

%%capture
import pandas as pd
# import modin.pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt
# import vaex # https://vaex.io/docs/index.html
import pathlib
from pathlib import *
import os
import pickle
# import cufflinks as cf
# import chart_studio.plotly as py
import seaborn as sns
import plotly.express as px
%matplotlib inline
from plotly.offline import download_plotlyjs, init_notebook_mode,plot,iplot

init_notebook_mode(connected=True)
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

In [ ]:
#### Common platedate - loading pkl file
with open("common_utrecht_eindhoven_platedate", "rb") as fp:
    common_utrecht_leek_platedate = pickle.load(fp)
    
with open("common_utrecht_eindhoven_numberplate", "rb") as fp:
    common_utrecht_leek_numberplate = pickle.load(fp)

Same Numberplates Other Locations within NL

In [ ]:
len(common_utrecht_leek_platedate)

In [ ]:
# df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\split_data\aos_summary_20M.csv',assume_missing=True,on_bad_lines='skip') # AOS SUMMARY

df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\df20M.csv',assume_missing=True,on_bad_lines='skip') # TRIP DETAIL


## Extracting all datapoints that pass through location1 and location 2 from TRIP_DETAIL 
df = df[df['platedate'].isin(common_utrecht_leek_platedate)]
with ProgressBar():
    df_trip_utrecht_leek = df.compute()
    
df_trip_utrecht_leek['Numberplate_copy'] = df_trip_utrecht_leek['Numberplate']
df_trip_utrecht_leek['platedate_copy'] = df_trip_utrecht_leek['platedate']
# len(df_trip_utrecht_leek['platedate_copy'].unique())

In [ ]:
# Function to extract indi dataframes (for each numberplate)
def dataframe_unique_numberplates(df):
    dfs = {}
    for platedate in df['platedate'].unique():
        try:
            dfs[platedate] = df.assign(**{platedate: df['platedate'].eq(platedate).astype(int)}).drop(columns='platedate')
        except:
            pass
        
    return dfs

df_trip_utrecht_leek=dataframe_unique_numberplates(df_trip_utrecht_leek)
# len(df_trip_utrecht_leek)
# df_trip_utrecht_leek.keys()
df_trip_utrecht_leek_keys=df_trip_utrecht_leek.keys()
# # df_trip_utrecht_leek_keys
df_trip_utrecht_leek_keys_list = list(df_trip_utrecht_leek.keys())
# print((df_trip_utrecht_leek_keys_list[0]))
# df_trip_utrecht_leek_keys

In [ ]:
print(type(df_trip_utrecht_leek))

In [ ]:
# @TODO ->  replace this key with a list of keys and run a for loop and save dfs dynamically

def extracting_df_per_numberplate(df,key):
    
    df=df.get(key)
    # df=df.get(df_trip_utrecht_leek_keys_list[0])
    df = df.drop(df[df[key]==0].index)
    return df
# df_utrecht_leek_1=extracting_df_per_numberplate(df_trip_utrecht_leek,df_trip_utrecht_leek_keys_list[0])

In [ ]:
# Filter trips between start and end location (right now we include trips from Location1 to Location2 and back).
# If you only want trips one way just comment the else statement

def extract_trip_between_start_end_location(df,start_location,end_location):
    df.drop(['index'],axis=1,inplace=True)
    df.reset_index(inplace=True)
    
    start_location_first_index=(df[df['Admin_area']==start_location].index)[0]
    end_location_first_index=(df[df['Admin_area']==end_location].index)[0]
    
    if start_location_first_index>end_location_first_index:
        df=df[end_location_first_index:start_location_first_index]
        return df
        
    else:
        df=df[start_location_first_index:end_location_first_index]
        return df

In [ ]:
location1='Utrecht' # Define Location 1 here
location2='Eindhoven' # Define Location 2 here

In [ ]:
## saving the dataframes as a list.
list_of_unique_dataframes=list()
for i in df_trip_utrecht_leek_keys_list:
    try:
        unique_dfs=extracting_df_per_numberplate(df_trip_utrecht_leek,i)
        unique_dfs_trip=extract_trip_between_start_end_location(unique_dfs,location1,location2) #location1,location2
        list_of_unique_dataframes.append(unique_dfs_trip)
    except:
        pass
# list_of_unique_dataframes

In [ ]:
df_utrecht_leek_filtered = pd.concat(list_of_unique_dataframes) #concatenating list to form dataframe

df_utrecht_leek_filtered=df_utrecht_leek_filtered.loc[:,~df_utrecht_leek_filtered.columns.str.startswith('B')] # dropping extra columns that start with B (platedate names)
df_utrecht_leek_filtered=df_utrecht_leek_filtered.loc[:,~df_utrecht_leek_filtered.columns.str.startswith('N')]# dropping extra columns that start with N (platedate names)
df_utrecht_leek_filtered

In [ ]:
# fig = px.scatter_mapbox(df_utrecht_leek_filtered,lat="Latitude",lon="Longitude",title="Utrecht-Leek  111 Trips (71 Trucks)")

# fig.update_layout(mapbox_style="open-street-map",margin={"r":0.75,"t":25,"l":0,"b":0},title_x=0.5)
# # fig.update_layout()
# fig.show()

In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\pickle_files\utrecht_eindhoven\trip_detail'
all_files = glob.glob(os.path.join(path, "*.pkl"))

df_utrecht_leek_filtered = pd.concat((pd.read_pickle(f) for f in all_files), ignore_index=True)

In [ ]:
df_utrecht_leek_filtered.to_pickle(r'D:\AOS FOT\Octo\pickle_files\utrecht_eindhoven\trip_detail_utrecht_eindhoven_20M.pkl')

Extracting platedates that pass through the same road and plugging them into complete dataframe to extract trips

In [ ]:
groupby_numberplate_utrecht_leek = df_utrecht_leek_filtered.groupby("platedate_copy")["KmTravelled"].sum()
groupby_numberplate_utrecht_leek

In [ ]:
#grouping df based on platedate_copy 
# (Note:Here you need to manually check the range of distance that's common and add that to next step so that we can extract the same road)

df_utrecht_leek_filtered_grouped=df_utrecht_leek_filtered.groupby('platedate_copy').sum() 
df_utrecht_leek_filtered_grouped

rslt_df = df_utrecht_leek_filtered_grouped.loc[(df_utrecht_leek_filtered_grouped['KmTravelled'] >= 70) & (df_utrecht_leek_filtered_grouped['KmTravelled'] <=90)] #Road 1 - (Utrecht-Eindhoven)


# rslt_df = df_utrecht_leek_filtered_grouped.loc[(df_utrecht_leek_filtered_grouped['KmTravelled'] >= 150) & (df_utrecht_leek_filtered_grouped['KmTravelled'] <=180)] #Road 1 - (Utrecht to Leek - Highway A27-A6-A7)
# rslt_df = df_utrecht_leek_filtered_grouped.loc[(df_utrecht_leek_filtered_grouped['KmTravelled'] >= 180) & (df_utrecht_leek_filtered_grouped['KmTravelled'] <=220)] #Road 2 - (Utrecht to Leek - Highway A28)

rslt_df=rslt_df.reset_index()
platedate_copy_same_road=rslt_df['platedate_copy']
platedate_copy_same_road # List of platedates that pass through the same road

In [ ]:
df=pd.read_csv(r"D:\AOS FOT\Octo\CSV Export\Individual Clustering Motorway\df_grp_aos_trip_detail_utrecht_leek_highway.csv")

In [ ]:
df = df_utrecht_leek_filtered[df_utrecht_leek_filtered['platedate_copy'].isin(platedate_copy_same_road)] # Extract complete df
df

In [ ]:
df['Numberplate'] = df['platedate_copy'].str[0:6]

In [ ]:
list_of_numberplate=list(df['Numberplate'])
len(list_of_numberplate)

In [ ]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar



# area_list = [location1,location2] # We can change these values to any start and end location
df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\TRIP_DETAIL.csv',delimiter=';', encoding='unicode_escape')

df = df[df['Numberplate'].isin(list_of_numberplate)]
df = df[df['Road_type']==1] #Motorway only trips
with ProgressBar():
    df = df.compute()
    
df['platedate'] = df['Numberplate'] + df['TNO_Time-stamp'].str[:10]

In [ ]:
df.to_csv(r"D:\AOS FOT\Octo\pickle_files\utrecht_leek\df_complete_aos_within_netherlands_motorways_utrecht_leek.csv")

In [ ]:

from shapely.geometry import Point, Polygon

# Create a Polygon
# coords_rotte = [(51.977901, 4.467480), (51.918861, 4.417012), (51.874370, 4.513485), (51.930857, 4.570123)]

coords_nl_belgium_de = [(51.319810, 3.199518), (51.141676, 4.044834), (51.151092, 4.433142), (51.160870, 4.911878),(51.119796, 5.379190),(50.916492, 5.347477),(50.732720, 5.558885),(50.646984, 5.868460),(50.682498, 6.225990),(50.774220, 6.788565

),(51.090741, 6.993645),(51.366330, 7.097475),(51.706392, 7.281298),(52.086558, 7.481535),(52.401805, 7.644418),(52.863343, 7.731970),(53.276982, 7.739262),(53.781184, 7.597500),(53.531285, 5.602048),(53.350568, 4.873847),(53.126881, 4.628008),(52.855526, 4.454778),(52.527262, 4.287084),(52.171565, 4.004032),(51.879936, 3.720315),(51.601678, 3.373136),
                (51.388379, 3.063189)]
poly_nl_belgium_de = Polygon(coords_nl_belgium_de)
poly_nl_belgium_de

In [ ]:
# initialize list of lists
from shapely.geometry import Point, Polygon
coords_nl_belgium_de = [(51.319810, 3.199518), (51.141676, 4.044834), (51.151092, 4.433142), (51.160870, 4.911878),(51.119796, 5.379190),(50.916492, 5.347477),(50.732720, 5.558885),(50.646984, 5.868460),(50.682498, 6.225990),(50.774220, 6.788565

),(51.090741, 6.993645),(51.366330, 7.097475),(51.706392, 7.281298),(52.086558, 7.481535),(52.401805, 7.644418),(52.863343, 7.731970),(53.276982, 7.739262),(53.781184, 7.597500),(53.531285, 5.602048),(53.350568, 4.873847),(53.126881, 4.628008),(52.855526, 4.454778),(52.527262, 4.287084),(52.171565, 4.004032),(51.879936, 3.720315),(51.601678, 3.373136),
                (51.388379, 3.063189)]
# Create the pandas DataFrame
df_poly_nl_belgium_de = pd.DataFrame(coords_nl_belgium_de, columns=['Lat', 'Lon'])
# print dataframe.
# df_poly_nl_belgium_de

In [ ]:
polygon = Polygon([tuple(x) for x in df_poly_nl_belgium_de[['Lat', 'Lon']].to_numpy()])
df['Within_NL'] = df.apply(lambda x: polygon.contains(Point(x['Latitude'], x['Longitude'])), axis=1)

In [ ]:
df_inter_city_trip_detail = df.drop(df[df['Within_NL']==False].index)
df_inter_city_trip_detail

In [ ]:
df_inter_city_trip_detail.to_csv(r"D:\AOS FOT\Octo\pickle_files\utrecht_leek\df_complete_td_within_netherlands_motorways_utrecht_leek.csv")

In [ ]:
df['Numberplate'] = df['platedate_copy'].str[0:6]

In [ ]:
len(df['platedate_copy'].unique())

In [ ]:
fig = px.scatter_mapbox(df[df['Numberplate']=='BVSG72'],lat="Latitude",lon="Longitude",title="Utrecht-Leek  111 Trips (71 Trucks)",color="Numberplate")

fig.update_layout(mapbox_style="open-street-map",margin={"r":0.75,"t":25,"l":0,"b":0},title_x=0.5)
# fig.update_layout()
fig.show()

In [ ]:
def pre_processing(df):
    # df = df.drop(df[df['Average_speed_fpp']==0].index) # Dropping rows where driver is still 
    df['Numberplate_copy'] = df['Numberplate'] # creating copy of numberplate
    # df['platedate_copy'] = df['platedate'] # creating copy of platedate
    return df

def creating_grps(df):
    """This function will split df into two groups to calculate mean speed 
    and total distance travelled by each numberplate in Urban areas within that city"""
    
    df_groupby_mean = df.groupby(['Numberplate']).mean()
    df_groupby_mean.reset_index(inplace=True)
    
    df_groupby_sum = df.groupby(['Numberplate']).sum()
    df_groupby_sum.reset_index(inplace=True)
    
    return df_groupby_mean,df_groupby_sum

def dropping_columns_mean(df):
    df=df.drop(['Unnamed: 0', 'index', 'Id', 'Header_Id',
             'Trip_Summary_Id', 'Latitude', 'Longitude', 'Status',
             'Meters_travelled', 'Time_elapsed','Average_speed_fpp',
             'Point_AOS_headway', 'Road_type', 'Road_class',
             'Road_form', 'Speed_restriction', 'TNO_Valid', 'TNO_Day',
             'TNO_Area_Valid', 'TNO_Time_elapsed', 'TNO_Average_speed',
             'TNO_Time-stamp_hour', 'TNO Trip Hour','Overspeeding_event',
             'KmTravelled'], axis = 1)
    return df 

def dropping_columns_sum(df):
    df=df.drop(['Unnamed: 0', 'index', 'Id', 'Header_Id',
             'Trip_Summary_Id', 'Latitude', 'Longitude', 'Status',
             'Meters_travelled', 'Time_elapsed', 'Average_speed_fpp',
             'Point_AOS_headway','Road_type', 'Road_class',
             'Road_form', 'Speed_restriction', 'TNO_Valid', 'TNO_Day',
             'TNO_Area_Valid', 'TNO_Time_elapsed', 'TNO_Average_speed',
             'TNO_Time-stamp_hour', 'TNO Trip Hour','Point_speed'], axis = 1)
    return df

In [ ]:
df_groupby_mean,df_groupby_sum = (df.pipe(pre_processing).pipe(creating_grps))
df_groupby_sum=dropping_columns_sum(df_groupby_sum)
df_groupby_mean=dropping_columns_mean(df_groupby_mean)


def merge(df_groupby_sum,df_groupby_mean):
    df = pd.merge(df_groupby_mean,df_groupby_sum, on='Numberplate', how='outer') #here Numberplate is common column
    df=df.rename(columns={'Point_speed': 'Point_speed_mean_intra_city',
                   'Overspeeding_event': 'Overspeeding_event_sum_intra_city',
                   'KmTravelled': 'KmTravelled_sum_intra_city'})
    df['norm_overspeeding']=df['Overspeeding_event_sum_intra_city']/df['KmTravelled_sum_intra_city']
    return df
    
df_city_grouped_trip_detail=merge(df_groupby_sum,df_groupby_mean)
df_city_grouped_trip_detail

In [ ]:
len(df['Numberplate'].unique())

In [ ]:
fig = px.scatter_mapbox(df,lat="Latitude",lon="Longitude",title="Utrecht-Leek  111 Trips (71 Trucks)")

fig.update_layout(mapbox_style="open-street-map",margin={"r":0.75,"t":25,"l":0,"b":0},title_x=0.5)
# fig.update_layout()
fig.show()

### **Trip Analysis** 

In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\trips_road_1'
all_files = glob.glob(os.path.join(path, "*.pkl"))

df = pd.concat((pd.read_pickle(f) for f in all_files), ignore_index=True)
df

In [ ]:
df

In [ ]:
df = df.drop(df[df['Numberplate']=='BTHN95'].index)
df = df.drop(df[df['Numberplate']=='BVFP92'].index)
df = df.drop(df[df['Numberplate']=='BTDS93'].index)
df = df.drop(df[df['Numberplate']=='BTFX65'].index)
df = df.drop(df[df['Numberplate']=='BTFX64'].index)
df = df.drop(df[df['Numberplate']=='BTFT45'].index)
# df = df.drop(df[df['Numberplate']=='BTHN95'].index)


fig = px.scatter_mapbox(df,lat="Latitude",lon="Longitude",color ="Numberplate",title="Utrecht-Leek  111 Trips (71 Trucks)")

fig.update_layout(mapbox_style="open-street-map",margin={"r":0.75,"t":25,"l":0,"b":0},title_x=0.5)
# fig.update_layout()
fig.show()

In [ ]:
df=df.loc[:,~df.columns.str.startswith('B')]
df=df.loc[:,~df.columns.str.startswith('N')]
df
# df_mean=df.groupby('platedate_copy').mean()
# df_mean

In [ ]:
df['Numberplate'] = df['platedate_copy'].str[0:6]
df

In [ ]:
len(df['Numberplate'].unique())

In [ ]:
df.to_csv(r"D:\AOS FOT\Octo\CSV Export\Individual Clustering Motorway\TripDetail\Utrecht_Leek\utrecht_leek_motorway_td.csv")

In [ ]:
ax=sns.displot(data=df_mean, x="Point_speed", kde=True,palette="Set2")
plt.xlabel('Mean Speed (km/h)')
plt.ylabel('Count')
plt.title('Distribution of Mean Speed of Trucks : Utrecht - Leek')


In [ ]:
plt.figure(figsize=(10,5))
flierprops = dict(marker='o', markerfacecolor='None', markersize=10,  markeredgecolor='black')

ax = sns.boxplot(data=df_mean, x="Point_speed",palette="Set2",orient="v",width=0.8,flierprops=flierprops)
ax = sns.swarmplot(x="Point_speed", data=df_mean, color=".3")
# notation indicating an outlier
ax.annotate('Outlier', xy=(89.2,0), xytext=(89,-0.2), fontsize=14,
            arrowprops=dict(arrowstyle='->', ec='grey', lw=2), bbox = dict(boxstyle="round", fc="0.8"))

# xtick, label, and title
# plt.xticks(fontsize=14)

plt.xlabel('Point Speed (km/h)')
plt.title('Distribution of Mean Speed of Trucks (Utrecht-Leek)')

In [ ]:
dfs = [y for x, y in df.groupby('platedate_copy', as_index=False)] # Splitting bigger dataframe into smaller dataframes using unique platedates 

for i in range(len(dfs)):
    dfs[i]['KmTravelled_cumsum'] = dfs[i]['KmTravelled'].cumsum()
dfs_cumsum = pd.concat(dfs)

In [ ]:
plt.figure(figsize=(20,10))
sns.set_theme()
dfs_cumsum = dfs_cumsum.drop(dfs_cumsum[dfs_cumsum['Meters_travelled']==0].index)

sns.lineplot(x='KmTravelled_cumsum',y='Point_speed',data=dfs_cumsum[dfs_cumsum['platedate_copy']=='BNZF662009-05-19'],linewidth=3,label="Highest Mean Speed",color='mediumslateblue')
sns.lineplot(x='KmTravelled_cumsum',y='Point_speed',data=dfs_cumsum[dfs_cumsum['platedate_copy']=='BNZF662009-01-20'],linewidth=3,color='red')

sns.lineplot(x='KmTravelled_cumsum',y='Point_speed',data=dfs_cumsum[dfs_cumsum['platedate_copy']=='BSLJ402008-12-06'],linewidth=3,color='coral')
sns.lineplot(x='KmTravelled_cumsum',y='Point_speed',data=dfs_cumsum[dfs_cumsum['platedate_copy']=='BSLJ402009-02-21'],linewidth=3,label="Lowest Mean Speed",color="mediumseagreen")

    
txt=(r"Vehicle Speed for drivers who drove the same route between Utrecht and Leek on the same road."
           "\n"  # Newline: the backslash is interpreted as usual
           r"The purple line represents the driver with highest mean speed among all trips. The green on the other hand represents the lowest mean speed."
           "\n" 
           r"Note : The green and orange represent the drivers that had the lower than average speed among all trips (Note:Same number plate possibly same driver (?))"
            "\n" 
           r"The purple and red represent the drivers that had the higher than average speed among all trips (Note:Same number plate possibly same driver (?))")

plt.figtext(0.5, -0.03, txt, wrap=True, horizontalalignment='center', fontsize=14)
plt.xlabel("Distance Travelled (km)", fontsize=14)
plt.ylabel("Point Speed (km/h)",fontsize=14)


In [ ]:
### Calculating Outliers

def mod_outlier(df):
    df1 = df.copy()
    df = df._get_numeric_data()


    q1 = df.quantile(0.25)
    q3 = df.quantile(0.75)

    iqr = q3 - q1

    lower_bound = q1 -(1.5 * iqr) 
    upper_bound = q3 +(1.5 * iqr)


    for col in col_vals:
        for i in range(0,len(df[col])):
            if df[col][i] < lower_bound[col]:            
                df[col][i] = lower_bound[col]

            if df[col][i] > upper_bound[col]:            
                df[col][i] = upper_bound[col]    


    for col in col_vals:
        df1[col] = df[col]

    return(df1)

# df = mod_outlier(df) # call function


#### **Extract Top 10% etc**

https://stackoverflow.com/questions/56710407/how-to-return-top-n-percent-of-pandas-dataframe-by-grouping

In [ ]:
x_largest=df_mean.nlargest(round(len(df_mean)/20), 'Point_speed')
x_largest.reset_index(inplace=True)
x_largest_platedate_copy=x_largest['platedate_copy']
x_largest_platedate_copy_list=x_largest_platedate_copy.tolist()
dfs_cumsum_top_10 = dfs_cumsum[dfs_cumsum['platedate_copy'].isin(x_largest_platedate_copy_list)]


In [ ]:
x_smallest=df_mean.nsmallest(round(len(df_mean)/20), 'Point_speed')
x_smallest.reset_index(inplace=True)
x_smallest_platedate_copy=x_smallest['platedate_copy']
x_smallest_platedate_copy_list=x_smallest_platedate_copy.tolist()
dfs_cumsum_bottom_10 = dfs_cumsum[dfs_cumsum['platedate_copy'].isin(x_smallest_platedate_copy_list)]

In [ ]:
plt.figure(figsize=(20,10))
sns.set_theme()
dfs_cumsum_top_10 = dfs_cumsum_top_10.drop(dfs_cumsum_top_10[dfs_cumsum_top_10['Meters_travelled']==0].index)
dfs_cumsum_bottom_10 = dfs_cumsum_bottom_10.drop(dfs_cumsum_bottom_10[dfs_cumsum_bottom_10['Meters_travelled']==0].index)

# sns.lineplot(x='KmTravelled_cumsum',y='Point_speed',data=dfs_cumsum_top_10,hue='platedate_copy',linewidth=2.5,legend=False)
sns.lineplot(x='KmTravelled_cumsum',y='Point_speed',data=dfs_cumsum_bottom_10,hue='platedate_copy',linewidth=2.5,legend=False)

# sns.lineplot(x='KmTravelled_cumsum',y='Point_speed',data=dfs_cumsum[dfs_cumsum['platedate_copy']=='BNZF662009-05-19'],linewidth=3,label="Highest Mean Speed",color='mediumslateblue')
# sns.lineplot(x='KmTravelled_cumsum',y='Point_speed',data=dfs_cumsum[dfs_cumsum['platedate_copy']=='BNZF662009-01-20'],linewidth=3,color='red')

# sns.lineplot(x='KmTravelled_cumsum',y='Point_speed',data=dfs_cumsum[dfs_cumsum['platedate_copy']=='BSLJ402008-12-06'],linewidth=3,color='coral')
# sns.lineplot(x='KmTravelled_cumsum',y='Point_speed',data=dfs_cumsum[dfs_cumsum['platedate_copy']=='BSLJ402009-02-21'],linewidth=3,label="Lowest Mean Speed",color="mediumseagreen")

    
# txt=(r"Vehicle Speed for drivers who drove the same route between Utrecht and Leek on the same road."
#            "\n"  # Newline: the backslash is interpreted as usual
#            r"The purple line represents the driver with highest mean speed among all trips. The green on the other hand represents the lowest mean speed."
#            "\n" 
#            r"Note : The green and orange represent the drivers that had the lower than average speed among all trips (Note:Same number plate possibly same driver (?))"
#             "\n" 
#            r"The purple and red represent the drivers that had the higher than average speed among all trips (Note:Same number plate possibly same driver (?))")

# plt.figtext(0.5, -0.03, txt, wrap=True, horizontalalignment='center', fontsize=14)
plt.xlabel("Distance Travelled (km)", fontsize=14)
plt.ylabel("Point Speed (km/h)",fontsize=14)


### Dropping all zero point speed

In [ ]:
df_drop_zero = df.drop(df[df['Point_speed']==0].index)
df_mean_wo_zero=df_drop_zero.groupby('platedate_copy').mean()

In [ ]:
x_largest=df_mean_wo_zero.nlargest(round(len(df_mean)/10), 'Point_speed')
x_largest.reset_index(inplace=True)
x_largest_platedate_copy=x_largest['platedate_copy']
x_largest_platedate_copy_list=x_largest_platedate_copy.tolist()
x_largest_mean=x_largest.groupby('platedate_copy').mean()
x_largest_mean['Type']='Top 10% Drivers'
# dfs_cumsum_top_10 = dfs_cumsum[dfs_cumsum['platedate_copy'].isin(x_largest_platedate_copy_list)]


In [ ]:
x_smallest=df_mean_wo_zero.nsmallest(round(len(df_mean)/10), 'Point_speed')
x_smallest.reset_index(inplace=True)
x_smallest_platedate_copy=x_smallest['platedate_copy']
x_smallest_platedate_copy_list=x_smallest_platedate_copy.tolist()
x_smallest_mean=x_smallest.groupby('platedate_copy').mean()
x_smallest_mean['Type']='Bottom 10% Drivers'
# dfs_cumsum_bottom_10 = dfs_cumsum[dfs_cumsum['platedate_copy'].isin(x_smallest_platedate_copy_list)]

In [ ]:
df_mean_wo_zero['Point_speed'].median()

In [ ]:
df_mean_wo_zero.reset_index(inplace=True)
df_mean_wo_zero

In [ ]:
x_median_vals = df_mean_wo_zero.loc[(df_mean_wo_zero['Point_speed'] >= 81.1) & (df_mean_wo_zero['Point_speed'] <=81.4)] #Road 1

In [ ]:
x_median_platedate_copy=x_median_vals['platedate_copy']
x_median_platedate_copy_list=x_median_platedate_copy.tolist()
# x_median_vals=x_median_vals.groupby('platedate_copy').mean()
x_median_vals['Type']='Mid 10% Drivers'
x_median_vals.reset_index(inplace=True)

In [ ]:
bottom_top_10=pd.concat([x_largest_mean, x_median_vals,x_smallest_mean])
bottom_top_10

In [ ]:
# plt.figure(figsize=(10,5))
sns.set_theme()


flierprops = dict(marker='o', markerfacecolor='None', markersize=10,  markeredgecolor='black')

ax = sns.boxplot(data=bottom_top_10, x="Type",y="Point_speed",palette="Set2",orient="v",width=0.8,flierprops=flierprops)
ax = sns.swarmplot(x="Type",y="Point_speed", data=bottom_top_10, color=".3")

plt.xlabel('Numberplates')
plt.ylabel('Mean Point Speed (km/h)')
plt.title('Speed of Trucks (Same Route : Utrecht-Leek)')

#### Checking if the drivers same drivers are slow/overspeed on other routes

In [ ]:
list_of_drivers_high_mean_speed=[]
for i in x_largest_platedate_copy_list:
    numberplate=i[:6]
    list_of_drivers_high_mean_speed.append(numberplate)
    


In [ ]:
list_of_drivers_middle_speed=[]
for i in x_median_platedate_copy_list:
    numberplate=i[:6]
    list_of_drivers_middle_speed.append(numberplate)
    
list_of_drivers_middle_speed

In [ ]:
(list_of_drivers_high_mean_speed)

### AOS Summary and Trip Detail of drivers overspeeding on Utrecht Leek Highway

In [ ]:
df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\df20M.csv',assume_missing=True, usecols=['Id', 'TNO_Time-stamp', 'Numberplate', 'Admin_area','Point_speed','Road_type_Rename'])

## Extracting all datapoints that pass through location1 and location 2 from TRIP_DETAIL 

df = df[df['Numberplate'].isin(list_of_drivers_high_mean_speed)]
with ProgressBar():
    df_trip_detail_utrecht_leek = df.compute()
# len(df_trip_utrecht_leek['platedate_copy'].unique())

In [ ]:
df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\df20M.csv',assume_missing=True, usecols=['Id', 'TNO_Time-stamp', 'Numberplate', 'Admin_area','Point_speed','Road_type_Rename'])

## Extracting all datapoints that pass through location1 and location 2 from TRIP_DETAIL 

df = df[df['Road_type_Rename'].isin(Road_type_Rename)]
with ProgressBar():
    df_aos_summary_utrecht_leek = df.compute()
# len(df_trip_utrecht_leek['platedate_copy'].unique())

In [ ]:
df=df[df['Numberplate']=='BNZF66']
fig = px.scatter_mapbox(df,lat="Latitude",lon="Longitude",color ="Numberplate",title="Utrecht-Leek  111 Trips (71 Trucks)")

fig.update_layout(mapbox_style="open-street-map",margin={"r":0.75,"t":25,"l":0,"b":0},title_x=0.5)
# fig.update_layout()
fig.show()

### Speed on Motorway

In [ ]:
df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\df20M.csv',assume_missing=True, usecols=['Id', 'TNO_Time-stamp', 'Numberplate', 'Admin_area','Point_speed','Road_type_Rename'])

## Extracting all datapoints that pass through location1 and location 2 from TRIP_DETAIL 
Road_type_Rename=['Motorway']
df = df.drop(df[df['Point_speed']==0].index)
df = df.drop(df[df['Point_speed']>150].index)
df = df[df['Road_type_Rename'].isin(Road_type_Rename)]
with ProgressBar():
    df = df.compute()
# len(df_trip_utrecht_leek['platedate_copy'].unique())

In [ ]:
df

In [ ]:
df = df.drop(df[df['Point_speed']==0].index)

In [ ]:
df = df.drop(df[df['Point_speed']>150].index)

In [ ]:
ax = sns.boxplot(x="Point_speed", hue='Road_type_Rename', data=df)

In [ ]:
list_of_drivers_lowest_mean_speed=[]
for i in x_smallest_platedate_copy_list:
    numberplate=i[:6]
    list_of_drivers_lowest_mean_speed.append(numberplate)

#### Extracting data for top 10% drivers

In [ ]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar


# area_list = ['Utrecht','Leek']
# df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\df20M.csv',delimiter=';', encoding='unicode_escape',usecols=['Id', 'TNO_Time-stamp', 'Numberplate', 'Admin_area','Point_speed','Latitude','Longitude'],low_memory=False)

df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\TRIP_DETAIL.csv',delimiter=';', encoding='unicode_escape')
# df
df = df[df['Numberplate'].isin(list_of_drivers_middle_speed)]
with ProgressBar():
    df = df.compute()

In [ ]:
df.to_pickle(r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\complete_df_mid_10_percent_drivers.pkl')

In [ ]:
df['Numberplate'].unique()

In [ ]:
df.columns

In [ ]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar


df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\TRIP_DETAIL.csv',delimiter=';', encoding='unicode_escape')
# df = df.reset_index()

# df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\TRIP_DETAIL.csv',assume_missing=True)
# df
df = df[df['Numberplate'].isin(list_of_drivers_lowest_mean_speed)]
with ProgressBar():
    df = df.compute()

In [ ]:
df.to_pickle(r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\list_of_drivers_lowest_mean_speed.pkl')

In [ ]:
len(list_of_drivers_lowest_mean_speed)

#### Reading Pickle Files

In [ ]:
df_top_Drivers=pd.read_pickle(r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\complete_df_top_10_percent_drivers.pkl')
df_top_Drivers=cast_to_correct_dtype(df_top_Drivers)
df_top_Drivers=rename_some_stuff(df_top_Drivers)

In [ ]:
# df_bottom=pd.read_pickle(r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\complete_df_bottom_10_percent_drivers.pkl')
# df=df.reindex()
# df_bottom=cast_to_correct_dtype(df_bottom)
# df_bottom=rename_some_stuff(df_bottom)

In [ ]:
complete_df_top_10_percent_drivers_urban=df_top_Drivers[df_top_Drivers['Road_type']==0]
complete_df_top_10_percent_drivers_motorway=df_top_Drivers[df_top_Drivers['Road_type']==1]

In [ ]:
complete_df_top_10_percent_drivers_motorway_grouped=complete_df_top_10_percent_drivers_urban.groupby("Numberplate").mean()
complete_df_top_10_percent_drivers_motorway_grouped['Type']="Top 10%"
complete_df_top_10_percent_drivers_motorway_grouped.reset_index(inplace=True)

In [ ]:
df_bottom_Drivers=pd.read_pickle(r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\complete_df_bottom_10_percent_drivers.pkl')
df_bottom_Drivers=cast_to_correct_dtype(df_bottom_Drivers)
df_bottom_Drivers=rename_some_stuff(df_bottom_Drivers)

In [ ]:
complete_df_bottom_10_percent_drivers_urban=df_bottom_Drivers[df_bottom_Drivers['Road_type']==0]
complete_df_bottom_10_percent_drivers_motorway=df_bottom_Drivers[df_bottom_Drivers['Road_type']==1]

In [ ]:
complete_df_bottom_10_percent_drivers_motorway_grouped=complete_df_bottom_10_percent_drivers_urban.groupby("Numberplate").mean()
complete_df_bottom_10_percent_drivers_motorway_grouped['Type']="Bottom 10 %"
complete_df_bottom_10_percent_drivers_motorway_grouped.reset_index(inplace=True)

In [ ]:
df_mid_Drivers=pd.read_pickle(r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\complete_df_mid_10_percent_drivers.pkl')
df_mid_Drivers=cast_to_correct_dtype(df_mid_Drivers)
df_mid_Drivers=rename_some_stuff(df_mid_Drivers)

In [ ]:
complete_df_mid_10_percent_drivers_urban=df_mid_Drivers[df_mid_Drivers['Road_type']==0]
complete_df_mid_10_percent_drivers_motorway=df_mid_Drivers[df_mid_Drivers['Road_type']==1]

In [ ]:
complete_df_mid_10_percent_drivers_motorway_grouped=complete_df_mid_10_percent_drivers_urban.groupby("Numberplate").mean()
complete_df_mid_10_percent_drivers_motorway_grouped['Type']="Mid 10 %"
complete_df_mid_10_percent_drivers_motorway_grouped.reset_index(inplace=True)

In [ ]:
complete_df_mid_10_percent_drivers_motorway_grouped

In [ ]:
list=[complete_df_top_10_percent_drivers_motorway_grouped,complete_df_mid_10_percent_drivers_motorway_grouped,complete_df_bottom_10_percent_drivers_motorway_grouped]
list=pd.concat(list)
# list

In [ ]:
# plt.figure(figsize=(10,5))
sns.set_theme()

flierprops = dict(marker='o', markerfacecolor='None', markersize=10,  markeredgecolor='black')

ax = sns.boxplot(data=list, x="Type",y="Point_speed",palette="Set2",orient="v",width=0.8,flierprops=flierprops)
ax = sns.swarmplot(x="Type",y="Point_speed", data=list, color=".3")

plt.xlabel('Numberplates')
plt.ylabel('Mean Point Speed (km/h)')
plt.title('Mean speed of groups in Urban Areas (All Trips)')



In [ ]:
import seaborn as sns
sns.set_theme(style="whitegrid")
tips = sns.load_dataset("tips")
ax = sns.boxplot(x=df["Point_speed"])

In [ ]:
df.columns

### Utrecht Leek Road 1 and Road 2 Analysis

### Road 1

In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\trips_road_1'
all_files = glob.glob(os.path.join(path, "*.pkl"))

df_road1 = pd.concat((pd.read_pickle(f) for f in all_files), ignore_index=True)
df_road1=df_road1.loc[:,~df_road1.columns.str.startswith('B')]
df_road1=df_road1.loc[:,~df_road1.columns.str.startswith('N')]
df_road1['Numberplate'] = df_road1['platedate_copy'].str[:6]
# df_mean=df.groupby('platedate_copy').mean()
# df_mean
df_road1.columns

#### Road 2

In [ ]:
path=r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\trips_road_2'
all_files = glob.glob(os.path.join(path, "*.pkl"))

df_road2 = pd.concat((pd.read_pickle(f) for f in all_files), ignore_index=True)
df_road2=df_road2.loc[:,~df_road2.columns.str.startswith('B')]
df_road2=df_road2.loc[:,~df_road2.columns.str.startswith('N')]
df_road2['Numberplate'] = df_road2['platedate_copy'].str[:6]
# df_road2

In [ ]:
len(df_road1['Numberplate'].unique())

In [ ]:
df=df_road1
df = df.drop(df[df['Numberplate']=='BTHN95'].index)
df = df.drop(df[df['Numberplate']=='BVFP92'].index)
df = df.drop(df[df['Numberplate']=='BTDS93'].index)
df = df.drop(df[df['Numberplate']=='BTFX65'].index)
df = df.drop(df[df['Numberplate']=='BTFX64'].index)
df = df.drop(df[df['Numberplate']=='BTFT45'].index)
df = df.drop(df[df['Numberplate']=='BTHN95'].index)


fig = px.scatter_mapbox(df,lat="Latitude",lon="Longitude",color ="Numberplate",title="Utrecht-Leek 131 Trips (82 Trucks) - Road 1")

fig.update_layout(mapbox_style="open-street-map",margin={"r":0.75,"t":25,"l":0,"b":0},title_x=0.5)
# fig.update_layout()
fig.show()

In [ ]:
df=df_road2
# df = df.drop(df[df['Numberplate']=='BSLJ40'].index)
# df = df.drop(df[df['Numberplate']=='BRHL10'].index)
# df = df.drop(df[df['Numberplate']=='BTFT45'].index)
# df = df.drop(df[df['Numberplate']=='BTFX65'].index)
# df = df.drop(df[df['Numberplate']=='BTFX64'].index)
# df = df.drop(df[df['Numberplate']=='BTFT45'].index)
# df = df.drop(df[df['Numberplate']=='BTD09'].index)

# df = df.drop(df[df['Numberplate']=='NNB298'].index)
# df = df.drop(df[df['Numberplate']=='BSNF22'].index)
# df = df.drop(df[df['Numberplate']=='BTDB31'].index)
# df = df.drop(df[df['Numberplate']=='BSLJ40'].index)
# df = df.drop(df[df['Numberplate']=='BSBJ50'].index)

# df = df.drop(df[df['Numberplate']=='BTND09'].index)
# df = df.drop(df[df['Numberplate']=='BRVV06'].index)
# df = df.drop(df[df['Numberplate']=='BSVH17'].index)
# df = df.drop(df[df['Numberplate']=='BTND05'].index)
# df = df.drop(df[df['Numberplate']=='BVFP93'].index)
# df = df.drop(df[df['Numberplate']=='BTDB37'].index)
# df = df.drop(df[df['Numberplate']=='BRJF18'].index)
# df = df.drop(df[df['Numberplate']=='NNB288'].index)
# df = df.drop(df[df['Numberplate']=='BRVZ288'].index)
# df = df.drop(df[df['Numberplate']=='BJSP23'].index)            
# df = df.drop(df[df['Numberplate']=='BTDB33'].index) 
# df = df.drop(df[df['Numberplate']=='BNSL21'].index) 

fig = px.scatter_mapbox(df,lat="Latitude",lon="Longitude",color ="Numberplate",title="Utrecht-Leek 131 Trips (82 Trucks) - Road 1")

fig.update_layout(mapbox_style="open-street-map",margin={"r":0.75,"t":25,"l":0,"b":0},title_x=0.5)
# fig.update_layout()
fig.show()

### Global Comaprison - Irrespective of location

All trips of a drivers who oversped on Highway between Utrecht and Leek

In [ ]:
df=pd.read_pickle(r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\complete_df_top_10_percent_drivers.pkl')
df=cast_to_correct_dtype(df)
df=rename_some_stuff(df)

Extracting trips for one driver

In [ ]:
df_trip_detail_overspeeding_driver1=df[df['Numberplate']==list_of_drivers_high_mean_speed[0]]
df_trip_detail_overspeeding_driver1.head()

#### Extracting AOS Summary for that driver

In [ ]:

fig = px.scatter_mapbox(aos_summary_overspeeding_driver1,lat="Latitude",lon="Longitude",color ="Event type Rename",title="AOS Events- Numberplate : BNZF66 ")

fig.update_layout(mapbox_style="open-street-map",margin={"r":0.75,"t":25,"l":0,"b":0},title_x=0.5)
# fig.update_layout()
fig.show()

In [ ]:
df=pd.read_pickle(r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\trip_detail_overspeeding_drivers\trip_detail_overspeeding_driver1.pkl')
df['Point_speed'].mean()

In [ ]:
df_no_zero = df.drop(df[(df['Point_speed']==0)].index)
df_no_zero

In [ ]:
df_no_zero['Point_speed'].mean()

In [ ]:
df_no_zero.columns

In [ ]:
df_no_zero['KmTravelled'].sum()

In [ ]:
fig = px.scatter_mapbox(df_no_zero,lat="Latitude",lon="Longitude",color ="Point_speed",title="All trips of overspeeding driver (BNZF66)")

fig.update_layout(mapbox_style="open-street-map",margin={"r":0.75,"t":25,"l":0,"b":0},title_x=0.5)
# fig.update_layout()
fig.show()

In [ ]:
aos_summary_overspeeding_driver1=pd.read_pickle(r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\aos_summary_overspeeding_drivers\aos_summary_overspeeding_driver1.pkl')
normalized_events_driver1=aos_summary_overspeeding_driver1['Event type Rename'].value_counts()/df_no_zero['KmTravelled'].sum()
normalized_events_driver1

In [ ]:
fig = px.scatter_mapbox(aos_summary_overspeeding_driver1,lat="Latitude",lon="Longitude",color ="Event type Rename",title="All Events Recorded for driver (BNZF66)")

fig.update_layout(mapbox_style="open-street-map",margin={"r":0.75,"t":25,"l":0,"b":0},title_x=0.5)
# fig.update_layout()
fig.show()

In [ ]:
normalized_events_driver1=normalized_events_driver1.to_frame() 
normalized_events_driver1

In [ ]:
normalized_events_driver1 = normalized_events_driver1.stack().reset_index()
sns.histplot(data=normalized_events_driver1,x=0)

### AOS SUMMARY of drivers that oversped on Utrecht-Leek Highway

In [ ]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar


# list_of_drivers_lowest_mean_speed=list_of_drivers_lowest_mean_speed
# area_list = [location1,location2] # We can change these values to any start and end location
df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\TRIP_DETAIL.csv',delimiter=';', encoding='unicode_escape')

df = df[df['Numberplate'].isin(list_of_drivers_lowest_mean_speed)]
with ProgressBar():
    df = df.compute()
    
df['platedate'] = df['Numberplate'] + df['TNO_Time-stamp'].str[:10]

In [ ]:
df=pd.read_pickle(r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\df_aos_summary_overspeeding.pkl')

In [ ]:
df_aos_summary_overspeeding_drivers = df.pipe(drop_first_row).pipe(resetIndex).pipe(cast_to_correct_dtype).pipe(rename_some_stuff).pipe(detect_overspeeding_count)
df_aos_summary_overspeeding_drivers.columns

In [ ]:
groupby_numberplate_overspeeding_drivers = df_aos_summary_overspeeding_drivers.groupby(['Numberplate', 'Event type Rename']).sum()
groupby_numberplate_overspeeding_drivers=groupby_numberplate_overspeeding_drivers.reset_index()
groupby_numberplate_overspeeding_drivers

In [ ]:
df_trip_detail_overspeeding_drivers=pd.read_pickle(r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\complete_df_top_10_percent_drivers.pkl')
df_trip_detail_overspeeding_drivers=df_trip_detail_overspeeding_drivers.pipe(drop_first_row).pipe(resetIndex).pipe(cast_to_correct_dtype).pipe(rename_some_stuff).pipe(detect_overspeeding_count)

In [ ]:
groupby_numberplate_overspeeding_drivers_trip_detail = df_trip_detail_overspeeding_drivers.groupby(['Numberplate']).sum()
groupby_numberplate_overspeeding_drivers_trip_detail=groupby_numberplate_overspeeding_drivers_trip_detail.reset_index()
groupby_numberplate_overspeeding_drivers_trip_detail['KM_travelled']=groupby_numberplate_overspeeding_drivers_trip_detail['Meters_travelled']/1000
groupby_numberplate_overspeeding_drivers_trip_detail['Overspeeding_event_by_distance_travelled']=groupby_numberplate_overspeeding_drivers_trip_detail['Overspeeding_event']/groupby_numberplate_overspeeding_drivers_trip_detail['KM_travelled']
groupby_numberplate_overspeeding_drivers_trip_detail

In [ ]:
sns.set_theme(style="whitegrid", palette="pastel")

ax=sns.barplot(data=groupby_numberplate_overspeeding_drivers_trip_detail,x="Numberplate",y="Overspeeding_event_by_distance_travelled")
ax.tick_params(axis='x', rotation=45)
plt.title(r"Normalized Overspeeding Count of drivers" "\n" r"that oversped on Utrecht Leek Highway")
ax.set_ylabel(r'$\dfrac{Over \ speeding \ Event}{Total \ Distance \ Covered \ (km)}$')

In [ ]:
df_trip_detail_lowpeeding_drivers=pd.read_pickle(r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\complete_df_bottom_10_percent_drivers.pkl')
df_trip_detail_lowspeeding_drivers=df_trip_detail_lowpeeding_drivers.pipe(drop_first_row).pipe(resetIndex).pipe(cast_to_correct_dtype).pipe(rename_some_stuff).pipe(detect_overspeeding_count)

In [ ]:
groupby_numberplate_lowspeeding_drivers_trip_detail=df_trip_detail_lowspeeding_drivers.groupby("Numberplate").sum()
groupby_numberplate_lowspeeding_drivers_trip_detail=groupby_numberplate_lowspeeding_drivers_trip_detail.reset_index()
groupby_numberplate_lowspeeding_drivers_trip_detail['KM_travelled']=groupby_numberplate_lowspeeding_drivers_trip_detail['Meters_travelled']/1000
groupby_numberplate_lowspeeding_drivers_trip_detail['Overspeeding_event_by_distance_travelled']=groupby_numberplate_lowspeeding_drivers_trip_detail['Overspeeding_event']/groupby_numberplate_lowspeeding_drivers_trip_detail['KM_travelled']
groupby_numberplate_lowspeeding_drivers_trip_detail

In [ ]:
df_trip_detail_midpeeding_drivers=pd.read_pickle(r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\complete_df_mid_10_percent_drivers.pkl')
df_trip_detail_midpeeding_drivers=df_trip_detail_midpeeding_drivers.pipe(drop_first_row).pipe(resetIndex).pipe(cast_to_correct_dtype).pipe(rename_some_stuff).pipe(detect_overspeeding_count)

In [ ]:
groupby_numberplate_midpeeding_drivers=df_trip_detail_midpeeding_drivers.groupby("Numberplate").sum()
groupby_numberplate_midpeeding_drivers=groupby_numberplate_midpeeding_drivers.reset_index()
groupby_numberplate_midpeeding_drivers['KM_travelled']=groupby_numberplate_midpeeding_drivers['Meters_travelled']/1000
groupby_numberplate_midpeeding_drivers['Overspeeding_event_by_distance_travelled']=groupby_numberplate_midpeeding_drivers['Overspeeding_event']/groupby_numberplate_midpeeding_drivers['KM_travelled']
groupby_numberplate_midpeeding_drivers

In [ ]:
groupby_numberplate_midpeeding_drivers['Type']="Mid 10%"

In [ ]:
# groupby_numberplate_lowspeeding_drivers_trip_detail['UniqueNumberplate']=groupby_numberplate_lowspeeding_drivers_trip_detail.groupby('Numberplate').ngroup()
groupby_numberplate_lowspeeding_drivers_trip_detail['Type']="Bottom 10%"

In [ ]:
# groupby_numberplate_overspeeding_drivers_trip_detail['UniqueNumberplate']=groupby_numberplate_overspeeding_drivers_trip_detail.groupby('Numberplate').ngroup()
groupby_numberplate_overspeeding_drivers_trip_detail['Type']="Top 10%"

In [ ]:
list=[groupby_numberplate_overspeeding_drivers_trip_detail,groupby_numberplate_midpeeding_drivers,groupby_numberplate_lowspeeding_drivers_trip_detail]
df_top_bottom=pd.concat(list)
df_top_bottom['UniqueNumberplate']=df_top_bottom.groupby('Numberplate').ngroup()

In [ ]:
df_top_bottom.reset_index(inplace=True)

In [ ]:
# plt.figure(figsize=(10,5))
sns.set_theme()
flierprops = dict(marker='o', markerfacecolor='None', markersize=2,  markeredgecolor='black')

ax = sns.boxplot(data=df_top_bottom, x="Type",y="Overspeeding_event_by_distance_travelled",palette="Set2",orient="v",width=0.8,flierprops=flierprops)
ax = sns.swarmplot(x="Type",y="Overspeeding_event_by_distance_travelled", data=df_top_bottom, color=".3")

plt.xlabel('Group')
plt.ylabel(r'$\dfrac{Over \ speeding \ Event}{Total \ Distance \ Covered \ (km)}$')
plt.title(r"Normalized Overspeeding Count of drivers" "\n" r" on Utrecht Leek Highway")


In [ ]:
# sns.set_theme(style="whitegrid", palette="pastel")

ax=sns.boxplot(data=df_top_bottom,y="Overspeeding_event_by_distance_travelled",x="Type")
# ax.tick_params(axis='x', rotation=45)
plt.title(r"Normalized Overspeeding Count of drivers" "\n" r" on Utrecht Leek Highway")
ax.set_ylabel(r'$\dfrac{Over \ speeding \ Event}{Total \ Distance \ Covered \ (km)}$')

In [ ]:
groupby_numberplate_overspeeding_drivers_headway_warning=groupby_numberplate_overspeeding_drivers[groupby_numberplate_overspeeding_drivers['Event type Rename']=='Headway Warning (long)']
sns.histplot(data=groupby_numberplate_overspeeding_drivers_headway_warning, x="Numberplate", y="Overspeeding_event")

In [ ]:
groupby_numberplate_overspeeding_drivers

#### Grouped Data based on Numberplates

In [ ]:

%%capture
import pandas as pd
# import modin.pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt
# import vaex # https://vaex.io/docs/index.html
import pathlib
from pathlib import *
import os
import pickle
# import cufflinks as cf
# import chart_studio.plotly as py
import seaborn as sns
import plotly.express as px
%matplotlib inline
from plotly.offline import download_plotlyjs, init_notebook_mode,plot,iplot

init_notebook_mode(connected=True)
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\grouped_numberplates_sum'
all_files = glob.glob(os.path.join(path, "*.pkl"))

df_grouped_complete = pd.concat((pd.read_pickle(f) for f in all_files), ignore_index=True)
df_grouped_complete

In [ ]:
df_grouped_complete=df_grouped_complete.groupby("Numberplate").sum()
df_grouped_complete['Overspeeding_event_by_distance_travelled']=df_grouped_complete['Overspeeding_event']/df_grouped_complete['KmTravelled']

In [ ]:
df_grouped_complete['Overspeeding_event_by_distance_travelled']=df_grouped_complete['Overspeeding_event']/df_grouped_complete['KmTravelled']

In [ ]:
df_grouped_complete

In [ ]:
plt.figure(figsize=(10,5))
flierprops = dict(marker='o', markerfacecolor='None', markersize=10,  markeredgecolor='black')

ax = sns.boxplot(data=df_grouped_complete, x="Overspeeding_event_by_distance_travelled",palette="Set2",orient="v",width=0.8,flierprops=flierprops)
# ax = sns.swarmplot(x="Overspeeding_event_by_distance_travelled", data=df_grouped_complete, color=".3")
# notation indicating an outlier
# ax.annotate('Outlier', xy=(89.2,0), xytext=(89,-0.2), fontsize=14,
#             arrowprops=dict(arrowstyle='->', ec='grey', lw=2), bbox = dict(boxstyle="round", fc="0.8"))

# xtick, label, and title
# plt.xticks(fontsize=14)

plt.xlabel(r'$\dfrac{Over \ speeding \ Event}{Total \ Distance \ Covered \ (km)}$')
plt.title('Normalized Overspeeding Events for all Numberplates')

In [ ]:
x_largest=df_grouped_complete.nlargest(round(len(df_grouped_complete)/20), 'Overspeeding_event_by_distance_travelled')
x_largest.reset_index(inplace=True)
x_largest['Type']="Top 10% Drivers"


In [ ]:
x_smallest=df_grouped_complete.nsmallest(round(len(df_grouped_complete)/20), 'Overspeeding_event_by_distance_travelled')
x_smallest.reset_index(inplace=True)
x_smallest['Type']="Bottom 10% Drivers"


In [ ]:
df_grouped_complete['Overspeeding_event_by_distance_travelled'].median()

In [ ]:
x_mid = df_grouped_complete.loc[(df_grouped_complete['Overspeeding_event_by_distance_travelled'] >= 0.1216) & (df_grouped_complete['Overspeeding_event_by_distance_travelled'] <=0.1325)] #Road 1
x_mid['Type']="Mid 10% Drivers"
x_mid.reset_index(inplace=True)
# len(rslt_df)

In [ ]:
df_list=[x_largest,x_mid,x_smallest]
df=pd.concat(df_list)

In [ ]:
# plt.figure(figsize=(10,5))
sns.set_theme()

flierprops = dict(marker='o', markerfacecolor='None', markersize=8,  markeredgecolor='black')

ax = sns.boxplot(data=df, x="Type",y="Overspeeding_event_by_distance_travelled",palette="Set2",orient="v",width=0.8,flierprops=flierprops)
ax = sns.swarmplot(x="Type",y="Overspeeding_event_by_distance_travelled", data=df, color=".3")

plt.xlabel('Drivers')
plt.ylabel(r'$\dfrac{Over \ speeding \ Event}{Total \ Distance \ Covered \ (km)}$')
plt.title('Normalized Overspeeding Events - Entire Dataset')

https://github.com/khuyentran1401/Data-science/blob/master/visualization/statsannotation_example.ipynb

Checking if drivers that have higher overspeeding events also have high mean speed

In [ ]:
list_of_x_largest=list(x_largest['Numberplate'])
list_of_x_mid=list(x_mid['Numberplate'])
list_of_x_smallest=list(x_smallest['Numberplate'])
# list_of_x_largest

### Extracting Trip Detail for drivers with high normalized overspeeding count

In [ ]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\AOS_SUMMARY.csv',delimiter=';', encoding='unicode_escape')

df = df[df['Numberplate'].isin(list_of_x_smallest)]
# df = df[df['Point_speed']==0]
with ProgressBar():
    df_aos_summary_overspeeding = df.compute()
    
df_aos_summary_overspeeding['platedate'] = df_aos_summary_overspeeding['Numberplate'] + df_aos_summary_overspeeding['TNO_Time-stamp'].str[:10]

In [ ]:
df_aos_summary_overspeeding.to_csv(r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\df_aos_summary_bottom_drivers.csv')

In [ ]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\AOS_SUMMARY.csv',delimiter=';', encoding='unicode_escape')

df = df[df['Numberplate'].isin(list_of_x_mid)]
# df = df[df['Point_speed']==0]
with ProgressBar():
    df_aos_summary_overspeeding = df.compute()
    
df_aos_summary_overspeeding['platedate'] = df_aos_summary_overspeeding['Numberplate'] + df_aos_summary_overspeeding['TNO_Time-stamp'].str[:10]

In [ ]:
df_aos_summary_overspeeding.to_csv(r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\df_aos_summary_mid_drivers.csv')

In [ ]:
# df_aos_summary_overspeeding.to_pickle(r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\df_aos_summary_overspeeding.pkl')

In [ ]:
all_trips_detail_of_drivers_that_have_high_normalized_overspeeding_event=pd.read_csv(r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\all_trips_detail_of_drivers_that_have_high_normalized_overspeeding_event.csv', encoding='unicode_escape',nrows=30000000)
all_trips_detail_of_drivers_that_have_high_normalized_overspeeding_event = all_trips_detail_of_drivers_that_have_high_normalized_overspeeding_event.drop(all_trips_detail_of_drivers_that_have_high_normalized_overspeeding_event[all_trips_detail_of_drivers_that_have_high_normalized_overspeeding_event['Point_speed']==0].index)
all_trips_detail_of_drivers_that_have_high_normalized_overspeeding_event=all_trips_detail_of_drivers_that_have_high_normalized_overspeeding_event[all_trips_detail_of_drivers_that_have_high_normalized_overspeeding_event['Road_type']==0]


In [ ]:
mean_speed_of_high_overspeed_normalized=all_trips_detail_of_drivers_that_have_high_normalized_overspeeding_event.groupby("Numberplate").mean()
mean_speed_of_high_overspeed_normalized=mean_speed_of_high_overspeed_normalized.reset_index()
mean_speed_of_high_overspeed_normalized.to_pickle(r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\mean_speed_of_high_overspeed_normalized_drivers_urban.pkl')

In [ ]:
sum_of_high_overspeed_normalized=all_trips_detail_of_drivers_that_have_high_normalized_overspeeding_event.groupby("Numberplate").sum()
sum_of_high_overspeed_normalized=sum_of_high_overspeed_normalized.reset_index()
sum_of_high_overspeed_normalized.to_pickle(r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\sum_of_high_overspeed_normalized_urban.pkl')

### Plotting

In [ ]:
mean_speed_of_high_overspeed_normalized=pd.read_pickle(r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\mean_speed_of_high_overspeed_normalized_drivers_urban.pkl')
mean_speed_of_high_overspeed_normalized

In [ ]:
mean_speed_of_high_overspeed_normalized.drop(['Unnamed: 0', 'Id', 'Header_Id', 'Trip_Summary_Id',
       'Latitude', 'Longitude', 'Status', 'Meters_travelled', 'Time_elapsed', 'Average_speed_fpp', 'Point_AOS_headway',
       'Number_of_lanes', 'Road_type', 'Road_class', 'Road_form',
       'Speed_restriction', 'TNO_Valid', 'TNO_Day', 'TNO_Area_Valid',
       'TNO_Time_elapsed', 'TNO_Average_speed'], axis = 1, inplace = True) 

In [ ]:
mean_speed_of_high_overspeed_normalized

In [ ]:
x_largest.columns

In [ ]:
x_largest.drop(['index', 'Id', 'Latitude', 'Longitude',
       'Meters_travelled', 'Average_speed_fpp', 'Number_of_lanes', 'Road_type',
       'Road_class', 'Road_form', 'Speed_restriction', 'TNO_Valid',
       'TNO_Time-stamp_hour', 'TNO Trip Hour', 'Overspeeding_event',
       'KmTravelled', 'Type'], axis = 1, inplace = True) 

In [ ]:
x_smallest_drivers_small=list(x_smallest['Numberplate'])

#### Merging columns together

In [ ]:
df_outer = pd.merge(x_largest, mean_speed_of_high_overspeed_normalized, on='Numberplate', how='outer')
df_outer

In [ ]:
sns.kdeplot(data=df_outer,x="Overspeeding_event_by_distance_travelled",y="Point_speed_y")
plt.title("Point Speed vs Normalized Overspeeding Events of drivers (Top 10%)")
plt.ylabel("Mean Point Speed (km/h)")
plt.xlabel(r'$\dfrac{Over \ speeding \ Event}{Total \ Distance \ Covered \ (km)}$')

In [ ]:
x_smallest_drivers_small

**AOS Summary of drivers with High/Low Normalized Overspeeding Event**

In [ ]:
df_bottom_overspeed_Drivers=pd.read__pickle(r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\df_aos_summary_low_speeding.pkl')

In [ ]:
complete_df_bottom_10_percent_drivers=pd.read_pickle(r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\complete_df_bottom_10_percent_drivers.pkl')
sum_speed_of_low_speed_normalized_trip_detail=complete_df_bottom_10_percent_drivers.groupby("Numberplate").sum()

In [ ]:
### Normalizing events values with distance

sum_high_overspeed_normalized=pd.read_pickle(r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\sum_speed_of_high_overspeed_normalized_trip_detail.pkl')
sum_high_overspeed_normalized['Km_travelled']=sum_high_overspeed_normalized['Meters_travelled']/1000
# sum_high_overspeed_normalized=sum_high_overspeed_normalized[sum_high_overspeed_normalized['Road_type']==0]


sum_high_overspeed_normalized.drop(['Unnamed: 0', 'Id', 'Header_Id', 'Trip_Summary_Id',
       'Latitude', 'Longitude', 'Status', 'Meters_travelled', 'Time_elapsed',
       'Point_speed', 'Average_speed_fpp', 'Point_AOS_headway',
       'Number_of_lanes', 'Road_type', 'Road_class', 'Road_form',
       'Speed_restriction', 'TNO_Valid', 'TNO_Day', 'TNO_Area_Valid',
       'TNO_Time_elapsed', 'TNO_Average_speed',], axis = 1, inplace = True)
sum_high_overspeed_normalized

In [ ]:
aos_summary_overspeeding=pd.read_csv(r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\df_aos_summary_overspeeding.csv')
aos_summary_overspeeding=rename_some_stuff(aos_summary_overspeeding)
# aos_summary_overspeeding_groupby=aos_summary_overspeeding.groupby("Numberplate").sum()
# aos_summary_overspeeding_groupby

In [ ]:
aos_summary_overspeeding.columns

In [ ]:
aos_summary_overspeeding_groupby=aos_summary_overspeeding.groupby(["Numberplate","Event type Rename"]).count()
aos_summary_overspeeding_groupby

In [ ]:
aos_summary_overspeeding_groupby=aos_summary_overspeeding_groupby.reset_index()

In [ ]:
aos_summary_overspeeding_groupby.columns

In [ ]:
# aos_summary_overspeeding_groupby_headway_warning=aos_summary_overspeeding_groupby[aos_summary_overspeeding_groupby['Event type Rename']=='Headway Warning (long)']
aos_summary_overspeeding_groupby.drop(['Unnamed: 0', 'Id', 'Header_Id',
       'AOS_event/action_date', 'AOS_event/action_time',
       'Latitude', 'Longitude', 'Event/action_speed', 'Number_of_lanes',
       'Road_type', 'Road_class', 'Road_form', 'Speed_restriction',
       'Admin_area', 'IMEI', 'VoucherID', 'TNO_Time-stamp', 'TNO_Valid',
       'TNO_Day', 'TNO_Area_Valid', 'platedate', 'Road_type_Rename'], axis = 1, inplace = True)

In [ ]:
aos_summary_overspeeding_groupby['Event type Rename'].unique()

In [ ]:
aos_summary_overspeeding_groupby_headway_warning=aos_summary_overspeeding_groupby[aos_summary_overspeeding_groupby['Event type Rename']=='Headway Warning (long)']
aos_summary_overspeeding_groupby_headway_warning_medium=aos_summary_overspeeding_groupby[aos_summary_overspeeding_groupby['Event type Rename']=='Headway Warning (medium)']
aos_summary_overspeeding_groupby_headway_warning_short=aos_summary_overspeeding_groupby[aos_summary_overspeeding_groupby['Event type Rename']=='Headway Warning (short)']
aos_summary_overspeeding_groupby_headway_warning_off=aos_summary_overspeeding_groupby[aos_summary_overspeeding_groupby['Event type Rename']=='Headway Warning = OFF']
aos_summary_overspeeding_groupby_lane_departure_warning_off=aos_summary_overspeeding_groupby[aos_summary_overspeeding_groupby['Event type Rename']=='Lane Departure Warning = OFF']
aos_summary_overspeeding_groupby_lane_departure_warning_on=aos_summary_overspeeding_groupby[aos_summary_overspeeding_groupby['Event type Rename']=='Left Lane Departure Warning = ON']
aos_summary_overspeeding_groupby_right_lane_on=aos_summary_overspeeding_groupby[aos_summary_overspeeding_groupby['Event type Rename']=='Right Lane Departure Warning = ON']
aos_summary_overspeeding_groupby_indicator_off=aos_summary_overspeeding_groupby[aos_summary_overspeeding_groupby['Event type Rename']=='Indicators = OFF']
aos_summary_overspeeding_groupby_brakes_on=aos_summary_overspeeding_groupby[aos_summary_overspeeding_groupby['Event type Rename']=='Brakes = ON']



aos_summary_overspeeding_groupby_headway_warning_list_numberplate=list(aos_summary_overspeeding_groupby_brakes_on['Numberplate'])

In [ ]:
df_grouped_complete.reset_index(inplace=True)

In [ ]:

df_grouped_complete = df_grouped_complete[df_grouped_complete['Numberplate'].isin(aos_summary_overspeeding_groupby_headway_warning_list_numberplate)]

In [ ]:
df_grouped_complete.drop(['index', 'Id', 'Latitude', 'Longitude',
       'Meters_travelled', 'Point_speed', 'Average_speed_fpp',
       'Number_of_lanes', 'Road_type', 'Road_class', 'Road_form',
       'Speed_restriction', 'TNO_Valid', 'TNO_Time-stamp_hour',
       'TNO Trip Hour', 'Overspeeding_event'], axis = 1, inplace = True)

In [ ]:
df_grouped_complete

In [ ]:
aos_summary_overspeeding_groupby_brakes_on

In [ ]:
df_outer = pd.merge(df_grouped_complete, aos_summary_overspeeding_groupby_brakes_on, on='Numberplate', how='outer')
df_outer['Event/action_type_distance_travelled']=df_outer['Event/action_type']/df_outer['KmTravelled']
df_outer

In [ ]:
sns.boxplot(data=df_outer["Event/action_type_distance_travelled"])

In [ ]:
sns.kdeplot(data=df_outer["Event/action_type_distance_travelled"])

In [ ]:
df_outer[df_outer['Event/action_type_distance_travelled']>6]['Numberplate']

#### Event vs AOS

In [ ]:
df_outer

In [ ]:
# df_outer = df_outer.drop(df_outer[df_outer['Numberplate']=='BSFP34'].index)
sns.set_theme()
sns.axes_style("darkgrid")
sns.kdeplot(data=df_outer,x="Event/action_type_distance_travelled",y="Overspeeding_event_by_distance_travelled", palette="crest")
plt.xlabel(r'$\dfrac{Brakes=ON \ Event}{Total \ Distance \ Covered \ (km)}$')
plt.ylabel(r'$\dfrac{Overspeeding \ Event}{Total \ Distance \ Covered \ (km)}$')
plt.title(r"$Normalized \ Overspeeding \ Event \ vs \ Braking \ Event$" "\n" r"$Numberplates (Top\ 10\% \ - \ Normalized \ Overspeeding \ Event)$" "\n" r"$Correlation \ Coefficient = 0.437$")

In [ ]:
# calculate Pearson's correlation
from scipy.stats import spearmanr
corr, _ = spearmanr(df_outer['Event/action_type_distance_travelled'], df_outer['Overspeeding_event_by_distance_travelled'])
print('Spearman correlation: %.3f' % corr)

**Distance Covered by each Numberplate**

In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\grouped_numberplates_sum'
all_files = glob.glob(os.path.join(path, "*.pkl"))

df_grouped_complete = pd.concat((pd.read_pickle(f) for f in all_files), ignore_index=True)
df_grouped_complete_trip_detail = df_grouped_complete.drop(df_grouped_complete[df_grouped_complete['Meters_travelled']==0].index)

In [ ]:
df_grouped_complete_trip_detail=df_grouped_complete_trip_detail.groupby("Numberplate").sum()
df_grouped_complete_trip_detail['Overspeeding_event_by_distance_travelled']=df_grouped_complete_trip_detail['Overspeeding_event']/df_grouped_complete_trip_detail['KmTravelled']
df_grouped_complete_trip_detail.reset_index(inplace=True)

In [ ]:
df_grouped_complete_trip_detail.drop(['index', 'Id', 'Latitude', 'Longitude',
       'Meters_travelled', 'Point_speed', 'Average_speed_fpp',
       'Number_of_lanes', 'Road_type', 'Road_class', 'Road_form',
       'Speed_restriction', 'TNO_Valid', 'TNO_Time-stamp_hour',
       'TNO Trip Hour'], axis = 1, inplace = True)
df_grouped_complete_trip_detail

#### AOS Event Sum per Numberplate

In [ ]:
aos_count_events=pd.read_pickle(r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\aos_summary_grouped\aos_count_events_complete.pkl')
aos_count_events['Event type Rename'].unique()


In [ ]:
aos_count_events_headway_long=aos_count_events[aos_count_events['Event type Rename']=='Headway Warning (long)']
aos_count_events_headway_long.drop(['Event type Rename'], axis = 1, inplace = True)
aos_count_events_headway_long.rename(columns={'Event/action_type': 'Count_headway_warning_long_count'},inplace=True, errors='raise')
aos_count_events_headway_long


In [ ]:
df_outer = pd.merge(df_grouped_complete_trip_detail, aos_count_events_headway_long, on='Numberplate', how='outer')
df_outer['Event/action_type_distance_travelled']=df_outer['Count_headway_warning_long_count']/df_outer['KmTravelled']
df_outer

In [ ]:
sns.set_style('whitegrid')
plt.figure(figsize=(10,5))
# sns.set_context('talk')
sns.set_palette(['#62C370', '#FFD166', '#EF476F'])


# sns.set_theme()
flierprops = dict(marker='o', markerfacecolor='None', markersize=6,  markeredgecolor='black')

ax = sns.boxplot(data=df_outer, x="Overspeeding_event_by_distance_travelled",palette="Set2",orient="v",width=0.8,flierprops=flierprops)
# ax = sns.swarmplot(data=df_outer,x="Overspeeding_event_by_distance_travelled", color=".3")

# plt.xlabel('Drivers')
plt.xlabel(r'$\dfrac{Over \ speeding \ Event}{Total \ Distance \ Covered \ (km)}$')
plt.title('Normalized Overspeeding Events - Entire Dataset')

In [ ]:
x_largest=df_outer.nlargest(round(len(df_outer)/20), 'Overspeeding_event_by_distance_travelled')
x_largest.reset_index(inplace=True)
x_largest['Type']="Top 10% Drivers"

In [ ]:
x_smallest=df_outer.nsmallest(round(len(df_outer)/20), 'Overspeeding_event_by_distance_travelled')
x_smallest.reset_index(inplace=True)
x_smallest['Type']="Bottom 10% Drivers"
df_list=[x_smallest,x_largest]
df=pd.concat(df_list)

In [ ]:
plt.figure(figsize=(10,5))
flierprops = dict(marker='o', markerfacecolor='None', markersize=2,  markeredgecolor='black')
df = df.drop(df[df.KmTravelled < 100].index)

ax = sns.boxplot(data=df, x="Type",y="Overspeeding_event_by_distance_travelled",palette="Set2",orient="v",width=0.8,flierprops=flierprops)
ax = sns.swarmplot(x="Type",y="Overspeeding_event_by_distance_travelled", data=df, color=".3")

plt.xlabel('Drivers')
plt.ylabel(r'$\dfrac{Over \ speeding \ Event}{Total \ Distance \ Covered \ (km)}$')
plt.title('Normalized Overspeeding Events - Entire Dataset')

In [ ]:
# aos_summary_overspeeding_driver1=pd.read_pickle(r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\aos_summary_overspeeding_drivers\aos_summary_overspeeding_driver1.pkl')
# normalized_events_driver1=aos_summary_overspeeding_driver1['Event type Rename'].value_counts()/df_no_zero['KmTravelled'].sum()
# normalized_events_driver1

### Drivers with high overspeeding event count

In [ ]:
list_of_drivers_low_speed=list(x_smallest['Numberplate'])
list_of_drivers_high_speed=list(x_largest['Numberplate'])

In [ ]:
# import dask.dataframe as dd
# from dask.diagnostics import ProgressBar


# df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\TRIP_DETAIL.csv',delimiter=';', encoding='unicode_escape')

# df = df[df['Numberplate'].isin(list_of_drivers_low_speed)]
# with ProgressBar():
#     df_low_overspeed_trip_detail = df.compute()
    
# df['platedate'] = df['Numberplate'] + df['TNO_Time-stamp'].str[:10]

In [ ]:
df_trip_detail_lowpeeding_drivers=pd.read_pickle(r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\complete_df_bottom_10_percent_drivers.pkl')


In [ ]:
df_trip_detail_lowpeeding_drivers_motorway=df_trip_detail_lowpeeding_drivers[df_trip_detail_lowpeeding_drivers['Road_type']==1]

In [ ]:
df_trip_detail_lowpeeding_drivers_urbnan=df_trip_detail_lowpeeding_drivers[df_trip_detail_lowpeeding_drivers['Road_type']==0]

In [ ]:
df_trip_detail_high_speed=pd.read_pickle(r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\mean_speed_of_high_overspeed_normalized_drivers.pkl')
df_trip_detail_high_speed.columns

In [ ]:
df_trip_detail_low_speed=pd.read_pickle(r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\df_low_overspeed_trip_detail.pkl')
df_trip_detail_low_speed.columns

In [ ]:
df_trip_detail_lowpeeding_drivers_urbnan_grp=df_trip_detail_lowpeeding_drivers_urbnan.groupby("Numberplate").mean()
df_trip_detail_lowpeeding_drivers_urbnan_grp.reset_index(inplace=True)

In [ ]:
df_trip_detail_lowpeeding_drivers_urbnan_grp.columns

In [ ]:
df_trip_detail_lowpeeding_drivers_urbnan_grp.drop(['Id', 'Header_Id', 'Trip_Summary_Id',
       'Latitude', 'Longitude', 'Status', 'Meters_travelled', 'Time_elapsed','Average_speed_fpp', 'Point_AOS_headway',
       'Number_of_lanes', 'Road_type', 'Road_class', 'Road_form',
       'Speed_restriction', 'TNO_Valid', 'TNO_Day', 'TNO_Area_Valid',
       'TNO_Time_elapsed', 'TNO_Average_speed'], axis = 1, inplace = True) 


In [ ]:
df_trip_detail_lowpeeding_drivers_urbnan_grp['Type']="Low Normalized Overspeeding Count - Urban"
df_trip_detail_lowpeeding_drivers_urbnan_grp

In [ ]:
df_trip_detail_low_speed_grouped.drop([ 'Speed_restriction', 'Meters_travelled'], axis = 1, inplace = True)
df_trip_detail_low_speed_grouped['Type']="Low Normalized Overspeeding Count"
df_trip_detail_low_speed_grouped

In [ ]:
list_of_dfs=[df_trip_detail_lowpeeding_drivers_urbnan_grp,df_trip_detail_high_speed]
list_of_dfs=pd.concat(list_of_dfs)
list_of_dfs

In [ ]:

# x_largest.drop(['index','Count_headway_warning_long_count', 'Event/action_type_distance_travelled', 'Type'], axis = 1, inplace = True) 
x_largest['Type']="High Normalized Overspeeding Count"

In [ ]:
# x_smallest.drop(['index','Count_headway_warning_long_count', 'Event/action_type_distance_travelled', 'Type'], axis = 1, inplace = True) 
x_smallest['Type']="Low Normalized Overspeeding Count"

In [ ]:
list_of_dfs_big_small=[x_smallest,x_largest]
list_of_dfs_big_small=pd.concat(list_of_dfs_big_small)
list_of_dfs_big_small

In [ ]:
df_outer = pd.merge(list_of_dfs, list_of_dfs_big_small, on='Numberplate', how='outer')
df_outer

In [ ]:
df_outer = df_outer.drop(df_outer[df_outer['KmTravelled'] < 100].index)

sns.scatterplot(x=df_outer['Overspeeding_event_by_distance_travelled'],y=df_outer['Point_speed'],s=60,hue=df_outer["Type_y"])
plt.ylabel(r"$Mean \ Point \ Speed \ (km/h)$")
plt.xlabel(r'$\dfrac{Over \ speeding \ Event}{Total \ Distance \ Covered \ (km)}$')
plt.title(r"$Mean \ Point \ Speed \ vs \ Normalized \ Over \ Speeding \ Event$")

In [ ]:
# calculate Pearson's correlation
from scipy.stats import spearmanr
corr, _ = spearmanr(x_largest['Overspeeding_event_by_distance_travelled'],df_trip_detail_high_speed['Point_speed'])
print('Spearman correlation: %.3f' % corr)

In [ ]:
x_smallest_drop = x_smallest.drop(x_smallest[x_smallest['Overspeeding_event_by_distance_travelled']<0.005].index)

sns.scatterplot(x=x_smallest_drop['Overspeeding_event_by_distance_travelled'],y=df_trip_detail_low_speed_grouped['Point_speed'],s=60)
plt.ylabel(r"$Mean \ Point \ Speed \ (km/h)$")
plt.xlabel(r'$\dfrac{Over \ speeding \ Event}{Total \ Distance \ Covered \ (km)}$')
plt.title(r"$Correlation \ Coefficient = -0.128$")

In [ ]:
df_trip_detail_high_speed

In [ ]:
all_trips_detail_of_drivers_that_have_high_normalized_overspeeding_event=pd.read_csv(r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\all_trips_detail_of_drivers_that_have_high_normalized_overspeeding_event.csv',nrows=20000000)
all_trips_detail_of_drivers_that_have_high_normalized_overspeeding_event

In [ ]:
df_trip_detail_lowpeeding_drivers['Road_type'].value_counts()/len(df_trip_detail_lowpeeding_drivers)*100

In [ ]:
d = {'Road Type':['Motorway', 'Urban', 'Extra Urban', 'Other'],
        'Percentage':[59.57, 34.25, 5.97, 0.20]}
pd.DataFrame(data=d)

In [ ]:
df_low_overspeed_trip_detail=pd.read_csv(r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\df_low_overspeed_trip_detail.csv',nrows=20000000)
df_low_overspeed_trip_detail

In [ ]:
(df_low_overspeed_trip_detail['Road_type'].value_counts())/len(df_low_overspeed_trip_detail)*100

In [ ]:
# calculate Pearson's correlation
from scipy.stats import spearmanr
corr, _ = spearmanr(x_smallest_drop['Overspeeding_event_by_distance_travelled'],df_trip_detail_low_speed_grouped['Point_speed'])
print('Spearman correlation: %.3f' % corr)

#### Entire Dataset (Mean Speed vs Overspeeding Count)

In [ ]:
# df_trip_detail=pd.read_pickle(r'D:\AOS FOT\Octo\pickle_files\utrecht_leek\df_trip_detail_for_grouping.pkl')
# df_trip_detail

In [ ]:
# dfReading the pickle file and assigning it to the variable df_trip_detail.
# # _trip_detail_groupby_sum=df_trip_detail.groupby("Numberplate").sum()
# df_trip_detail_groupby_mean=df_trip_detail.groupby("Numberplate").mean()

### Overspeeding Event based on Type of Road

### Split the entire dataset based on type of road

In [ ]:
complete_df_top_10_percent_drivers_urban=df[df['Road_type']==0]
complete_df_top_10_percent_drivers_motorway=df[df['Road_type']==1]

In [ ]:
df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\TRIP_DETAIL.csv',delimiter=';', encoding='unicode_escape')

## Extracting all datapoints that pass through location1 and location 2 from TRIP_DETAIL 
# complete_df_top_10_percent_drivers_motorway=df[df['Road_type']==1]
df = df[df['Road_type']==1]
with ProgressBar():
    df__trip_detail_motorway_20M = df.compute()
# len(df_trip_utrecht_leek['platedate_copy'].unique())

## **Splitting Entire Dataset into Urban and Motorway**

#### Extracting Urban Area - Trip Detail

In [ ]:

%%capture
import pandas as pd
# import modin.pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt
# import vaex # https://vaex.io/docs/index.html
import pathlib
from pathlib import *
import os
import pickle
# import cufflinks as cf
# import chart_studio.plotly as py
import seaborn as sns
import plotly.express as px
%matplotlib inline
from plotly.offline import download_plotlyjs, init_notebook_mode,plot,iplot

init_notebook_mode(connected=True)
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

### **Mean speed of drivers in diff Urban areas**

#### **Amsterdam**

In [ ]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar


# area_list = ['Utrecht','Leek']
# df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\df20M.csv',delimiter=';', encoding='unicode_escape',usecols=['Id', 'TNO_Time-stamp', 'Numberplate', 'Admin_area','Point_speed','Latitude','Longitude'],low_memory=False)

df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\split_data\aos_summary_140M.csv', encoding='unicode_escape',low_memory=False,assume_missing=True)
# df
df = df[df['Admin_area']=='Rotterdam']
with ProgressBar():
    df = df.compute()

In [ ]:
df.to_csv(r'D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\split_data\Urban\Rotterdam\AOS_SUMMARY_140M_urban_rotterdam.csv')

In [ ]:
df.to_pickle(r'D:\AOS FOT\Octo\pickle_files\Complete_data_aos_summary\Urban\Rotterdam\AOS_SUMMARY_140M_urban_rotterdam.pkl')

In [ ]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar


# area_list = ['Utrecht','Leek']
# df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\df20M.csv',delimiter=';', encoding='unicode_escape',usecols=['Id', 'TNO_Time-stamp', 'Numberplate', 'Admin_area','Point_speed','Latitude','Longitude'],low_memory=False)

df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\split_data\aos_summary_120M.csv', encoding='unicode_escape',low_memory=False,assume_missing=True)
# df
df = df[df['Admin_area']=='Rotterdam']
with ProgressBar():
    df = df.compute()

In [ ]:
df.to_csv(r'D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\split_data\Urban\Rotterdam\AOS_SUMMARY_120M_urban_rotterdam.csv')

In [ ]:
df.to_pickle(r'D:\AOS FOT\Octo\pickle_files\Complete_data_aos_summary\Urban\Rotterdam\AOS_SUMMARY_120M_urban_rotterdam.pkl')

In [ ]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar


# area_list = ['Utrecht','Leek']
# df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\df20M.csv',delimiter=';', encoding='unicode_escape',usecols=['Id', 'TNO_Time-stamp', 'Numberplate', 'Admin_area','Point_speed','Latitude','Longitude'],low_memory=False)

df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Urban\df_trip_detail_240M_urban.csv', encoding='unicode_escape',low_memory=False)
# df
df = df[df['Admin_area']=='Zwolle']
with ProgressBar():
    df = df.compute()

In [ ]:
df.to_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Urban\Zwolle\TRIP_DETAIL_240M_urban_zwolle.csv')

In [ ]:
df.to_pickle(r'D:\AOS FOT\Octo\pickle_files\Complete_data_Trip_detail\Urban\Zwolle\TRIP_DETAIL_240M_urban_zwolle.pkl')

### Filtering data

In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\pickle_files\Complete_data_Trip_detail\Urban\Zwolle'
all_files = glob.glob(os.path.join(path, "*.pkl"))

df_ams = pd.concat((pd.read_pickle(f) for f in all_files), ignore_index=True)

In [ ]:
df_ams_fil = df_ams.drop(df_ams[df_ams['Average_speed_fpp']==0].index)
df_ams_fil['Numberplate_copy'] = df_ams_fil['Numberplate']
df_ams_fil['platedate_copy'] = df_ams_fil['platedate']

In [ ]:
df_ams_fil.columns

In [ ]:
df_ams_fil

In [ ]:
df_ams_fil_numberplate_groupby_mean = df_ams_fil.groupby(['Numberplate']).mean()
df_ams_fil_numberplate_groupby_mean.reset_index(inplace=True)
df_ams_fil_numberplate_groupby_sum = df_ams_fil.groupby(['Numberplate']).sum()
df_ams_fil_numberplate_groupby_sum.reset_index(inplace=True)


#### Dropping columns that are not present in both

In [ ]:
df_ams_fil_numberplate_groupby_mean.drop(['Unnamed: 0.1', 'Unnamed: 0', 'index', 'Id', 'Header_Id',
       'Trip_Summary_Id', 'Latitude', 'Longitude', 'Status',
       'Meters_travelled', 'Time_elapsed','Average_speed_fpp',
       'Point_AOS_headway', 'Number_of_lanes', 'Road_type', 'Road_class',
       'Road_form', 'Speed_restriction', 'TNO_Valid', 'TNO_Day',
       'TNO_Area_Valid', 'TNO_Time_elapsed', 'TNO_Average_speed',
       'TNO_Time-stamp_hour', 'TNO Trip Hour','Overspeeding_event',
       'KmTravelled'], axis = 1, inplace = True) 


In [ ]:
df_ams_fil_numberplate_groupby_sum.drop(['Unnamed: 0.1', 'Unnamed: 0', 'index', 'Id', 'Header_Id',
       'Trip_Summary_Id', 'Latitude', 'Longitude', 'Status',
       'Meters_travelled', 'Time_elapsed', 'Point_speed', 'Average_speed_fpp',
       'Point_AOS_headway', 'Number_of_lanes', 'Road_type', 'Road_class',
       'Road_form', 'Speed_restriction', 'TNO_Valid', 'TNO_Day',
       'TNO_Area_Valid', 'TNO_Time_elapsed', 'TNO_Average_speed',
       'TNO_Time-stamp_hour', 'TNO Trip Hour'], axis = 1, inplace = True) 


In [ ]:
df_ams_grouped = pd.merge(df_ams_fil_numberplate_groupby_mean, df_ams_fil_numberplate_groupby_sum, on='Numberplate', how='outer') #here id is common column
df_ams_grouped['norm_overspeeding']=df_ams_grouped['Overspeeding_event']/df_ams_grouped['KmTravelled']
df_ams_grouped

In [ ]:
sns.scatterplot(data=df_ams_grouped[df_ams_grouped['KmTravelled']<200],y="Point_speed",x="norm_overspeeding")

In [ ]:
df_ams_grouped['KmTravelled'].median()

In [ ]:
df=df_ams_grouped[df_ams_grouped['KmTravelled']<2000]

In [ ]:
sns.boxplot(data=df,x='KmTravelled')
plt.title('Total Distance Covered by different Numberplates over all trips')

In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\pickle_files\Complete_data_aos_summary\Urban\Zwolle'
all_files = glob.glob(os.path.join(path, "*.pkl"))

df_ams_aos = pd.concat((pd.read_pickle(f) for f in all_files), ignore_index=True)
df_ams_aos_urban=df_ams_aos[df_ams_aos['Road_type_Rename']=='Urban']
# df_ams_aos_urban

In [ ]:
df_ams_aos_urban_ground=df_ams_aos_urban.groupby(['Numberplate','Event type Rename']).size().unstack(fill_value=0)
df_ams_aos_urban_ground.reset_index(inplace=True)
df_ams_aos_urban_ground.columns

In [ ]:
df_ams_grouped_trip_aos = pd.merge(df_ams_grouped, df_ams_aos_urban_ground, on='Numberplate', how='outer') #here id is common column
# df_ams_grouped['norm_overspeeding']=df_ams_grouped['Overspeeding_event']/df_ams_grouped['KmTravelled']
df_ams_grouped_trip_aos

In [ ]:
df_ams_grouped_trip_aos_drop_nan = df_ams_grouped_trip_aos.dropna(axis = 0, how = 'any')
df_ams_grouped_trip_aos_drop_nan

In [ ]:
df_ams_grouped_trip_aos_drop_nan.columns

In [ ]:
%%capture
df_ams_grouped_trip_aos_drop_nan = df_ams_grouped_trip_aos.dropna(axis = 0, how = 'any')
df_ams_grouped_trip_aos_drop_nan

df_ams_grouped_trip_aos_drop_nan['norm_Indicators = OFF']=df_ams_grouped_trip_aos_drop_nan[' Indicators = OFF']/df_ams_grouped_trip_aos_drop_nan['KmTravelled']
df_ams_grouped_trip_aos_drop_nan['norm_brakes = OFF']=df_ams_grouped_trip_aos_drop_nan['Brakes = OFF']/df_ams_grouped_trip_aos_drop_nan['KmTravelled']
df_ams_grouped_trip_aos_drop_nan['norm_brakes = ON']=df_ams_grouped_trip_aos_drop_nan['Brakes = ON']/df_ams_grouped_trip_aos_drop_nan['KmTravelled']
df_ams_grouped_trip_aos_drop_nan['norm_headway_warning(long)']=df_ams_grouped_trip_aos_drop_nan['Headway Warning (long)']/df_ams_grouped_trip_aos_drop_nan['KmTravelled']
df_ams_grouped_trip_aos_drop_nan['norm_headway_warning(medium)']=df_ams_grouped_trip_aos_drop_nan['Headway Warning (medium)']/df_ams_grouped_trip_aos_drop_nan['KmTravelled']
df_ams_grouped_trip_aos_drop_nan['norm_headway_warning(short)']=df_ams_grouped_trip_aos_drop_nan['Headway Warning (short)']/df_ams_grouped_trip_aos_drop_nan['KmTravelled']
df_ams_grouped_trip_aos_drop_nan['norm_Headway Warning = OFF']=df_ams_grouped_trip_aos_drop_nan['Headway Warning = OFF']/df_ams_grouped_trip_aos_drop_nan['KmTravelled']
df_ams_grouped_trip_aos_drop_nan['norm_Lane Departure Warning = OFF']=df_ams_grouped_trip_aos_drop_nan['Lane Departure Warning = OFF']/df_ams_grouped_trip_aos_drop_nan['KmTravelled']
df_ams_grouped_trip_aos_drop_nan['norm_Left Indicator = ON']=df_ams_grouped_trip_aos_drop_nan['Left Indicator = ON']/df_ams_grouped_trip_aos_drop_nan['KmTravelled']
df_ams_grouped_trip_aos_drop_nan['norm_Left Lane Departure Warning = ON']=df_ams_grouped_trip_aos_drop_nan['Left Lane Departure Warning = ON']/df_ams_grouped_trip_aos_drop_nan['KmTravelled']
df_ams_grouped_trip_aos_drop_nan['norm_Left and Right Indicator = ON']=df_ams_grouped_trip_aos_drop_nan['Left and Right Indicator = ON']/df_ams_grouped_trip_aos_drop_nan['KmTravelled']
df_ams_grouped_trip_aos_drop_nan['norm_Right Indicator = ON']=df_ams_grouped_trip_aos_drop_nan['Right Indicator = ON']/df_ams_grouped_trip_aos_drop_nan['KmTravelled']
df_ams_grouped_trip_aos_drop_nan['norm_Right Lane Departure Warning = ON']=df_ams_grouped_trip_aos_drop_nan['Right Lane Departure Warning = ON']/df_ams_grouped_trip_aos_drop_nan['KmTravelled']


In [ ]:
sns.boxplot(df_ams_grouped_trip_aos_drop_nan["Point_speed"])

In [ ]:
sns.scatterplot(y=df_ams_grouped_trip_aos_drop_nan["norm_brakes = ON"],x=df_ams_grouped_trip_aos_drop_nan["Point_speed"])

### Splitting Dataset into Top/Mid/Bottom groups

In [ ]:
df_ams_grouped_trip_aos_drop_nan

In [ ]:
df_ams_grouped_trip_aos_drop_nan[[' Indicators = OFF', 'Brakes = OFF', 'Brakes = ON',
       'Headway Warning (long)', 'Headway Warning (medium)',
       'Headway Warning (short)', 'Headway Warning = OFF',
       'Lane Departure Warning = OFF', 'Left Indicator = ON',
       'Left Lane Departure Warning = ON', 'Left and Right Indicator = ON',
       'Right Indicator = ON', 'Right Lane Departure Warning = ON']].div(df_ams_grouped_trip_aos_drop_nan['KmTravelled'], axis=0)

In [ ]:
sns.histplot(data=df_ams_grouped_trip_aos_drop_nan, x="KmTravelled", kde=True,stat="probability")
plt.xlabel("Distance Travelled (km)")
plt.title("Distance Covered by each Numberplate in Ams (Urban)")
plt.ylabel("Relative Frequency")

In [ ]:
sns.kdeplot(data=df_ams_grouped_trip_aos_drop_nan, x="KmTravelled")
plt.xlabel("Distance Travelled (km)")
plt.title("Distance Covered by each Numberplate in Ams (Urban)")
# plt.ylabel("Relative Frequency")

#### Detecting Outliers

In [ ]:
def remove_outlier_IQR(df):
    Q1=df.quantile(0.25)
    Q3=df.quantile(0.75)
    IQR=Q3-Q1
    df_final=df[~((df<(Q1-1.5*IQR)) | (df>(Q3+1.5*IQR)))]
    return df_final

In [ ]:
rem_df_ams_grp_km_travelled=remove_outlier_IQR(df_ams_grouped_trip_aos_drop_nan["KmTravelled"])
len(rem_df_ams_grp_km_travelled)

In [ ]:
sns.histplot(rem_df_ams_grp_km_travelled,kde=True)
plt.xlabel("Distance Travelled (km)")
plt.title("Distance Covered by each Numberplate in Ams (Urban)")

#### Removing bottom 10% of data cause that will not be accurate representation of a drivers behaviour

In [ ]:
x_smallest_ams_urban=df_ams_grouped_trip_aos_drop_nan.nsmallest(round(len(df_ams_grouped_trip_aos_drop_nan)/10),'KmTravelled')
x_smallest_ams_urban.reset_index(inplace=True)
# x_smallest_ams_urban

x_smallest_ams_urban_numberplate=x_smallest_ams_urban['Numberplate']

In [ ]:
x_smallest_ams_urban_numberplate

In [ ]:
x_largest_ams_urban=df_ams_grouped_trip_aos_drop_nan.nlargest(round(len(df_ams_grouped_trip_aos_drop_nan)/20),'KmTravelled')
x_largest_ams_urban.reset_index(inplace=True)
# x_largest_ams_urban
x_largest_ams_urban_numberplate=x_largest_ams_urban['Numberplate']

In [ ]:
x_largest_ams_urban_numberplate

In [ ]:
df = df_ams_grouped_trip_aos_drop_nan[~df_ams_grouped_trip_aos_drop_nan['Numberplate'].isin(x_largest_ams_urban_numberplate)]
df = df[~df['Numberplate'].isin(x_smallest_ams_urban_numberplate)]
df

In [ ]:
df['KmTravelled'].min()

In [ ]:
sns.histplot(df['KmTravelled'],kde=True)
plt.xlabel("Distance Travelled (km)")
plt.title("Distance Covered by each Numberplate in Ams (Urban)")

#### Splitting grp into top/mid/bottom based on Overspeeding

In [ ]:
df.columns

In [ ]:
x_smallest_ams_urban_overspeeding=df.nsmallest(round(len(df)/10),'norm_overspeeding')
x_smallest_ams_urban_overspeeding.reset_index(inplace=True)
# x_smallest_ams_urban
x_smallest_ams_urban_overspeeding_numberplate=x_smallest_ams_urban_overspeeding['Numberplate']

In [ ]:
x_largest_ams_urban_overspeeding=df.nlargest(round(len(df)/10),'norm_overspeeding')
x_largest_ams_urban_overspeeding.reset_index(inplace=True)
# x_smallest_ams_urban
x_largest_ams_urban_overspeeding_numberplate=x_largest_ams_urban_overspeeding['Numberplate']

In [ ]:
len(x_largest_ams_urban_overspeeding_numberplate)

In [ ]:
df['norm_overspeeding'].median()

In [ ]:
x_median_vals = df.loc[(df['norm_overspeeding'] >= 0.0298) & (df['norm_overspeeding'] <=0.0342)] #Road 1
len(x_median_vals)

In [ ]:
x_median_vals = df.loc[(df['norm_overspeeding'] >= 0.0298) & (df['norm_overspeeding'] <=0.0342)] #Road 1
x_median_vals.reset_index(inplace=True)
x_median_ams_urban_overspeeding_numberplate=x_median_vals['Numberplate']

### Extracting All trips of overspeeding drivers - Largest

In [ ]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar


df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\AOS_SUMMARY.csv',encoding='unicode_escape',delimiter=';')

df = df[df['Road_type']==0]
df = df[df['Numberplate'].isin(x_median_ams_urban_overspeeding_numberplate)]
with ProgressBar():
    df = df.compute()
    
df['platedate'] = df['Numberplate'] + df['TNO_Time-stamp'].str[:10]

# df.to_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Urban\Amsterdam\Complete Trip Detail Drivers Norm Overspeeding Amsterdam\Bottom\df_complete_trip_detail_260M.csv')

In [ ]:
df.to_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Urban\Zwolle\Complete AOS Summary Drivers Norm Overspeeding Zwolle\df_complete_aos_summary_overspeeding_zwolle_mid.csv')

In [ ]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar


df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\AOS_SUMMARY.csv',encoding='unicode_escape',delimiter=';')

df = df[df['Road_type']==0]
df = df[df['Numberplate'].isin(x_largest_ams_urban_overspeeding_numberplate)]
with ProgressBar():
    df = df.compute()
    
df['platedate'] = df['Numberplate'] + df['TNO_Time-stamp'].str[:10]

# df.to_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Urban\Amsterdam\Complete Trip Detail Drivers Norm Overspeeding Amsterdam\Bottom\df_complete_trip_detail_260M.csv')

In [ ]:
df.to_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Urban\Zwolle\Complete AOS Summary Drivers Norm Overspeeding Zwolle\df_complete_aos_summary_overspeeding_zwolle_top.csv')

In [ ]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar


df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\AOS_SUMMARY.csv',encoding='unicode_escape',delimiter=';')

df = df[df['Road_type']==0]
df = df[df['Numberplate'].isin(x_smallest_ams_urban_overspeeding_numberplate)]
with ProgressBar():
    df = df.compute()
    
df['platedate'] = df['Numberplate'] + df['TNO_Time-stamp'].str[:10]

# df.to_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Urban\Amsterdam\Complete Trip Detail Drivers Norm Overspeeding Amsterdam\Bottom\df_complete_trip_detail_260M.csv')

In [ ]:
df.to_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Urban\Zwolle\Complete AOS Summary Drivers Norm Overspeeding Zwolle\df_complete_aos_summary_overspeeding_zwolle_bottom.csv')